# Programación Entera: Heurísticas para el Problema de la Mochila

#### Descripción

TODO

#### Autor
  
  * Sergio García Prado - [garciparedes.me](https://garciparedes.me)
  
#### Fecha

  * Mayo de 2018


## Contenidos
  * [Introducción](#Introducción)
  * [Problema de la Mochila](#Problema-de-la-Mochila)
    * [Mochila Simple](#Mochila-Simple)
    * [Mochila Múltiple](#Mochila-Múltiple)
    * [Mochila Multidimensional](#Mochila-Multidimensional)
  * [Problema de Empaquetamiento](#Problema-de-Empaquetamiento)
  * [Conclusiones](#Conclusiones)

## Introducción

TODO

#### Configuración del Entorno

In [1]:
rm(list = ls())

In [2]:
library(slam)
library(Rglpk)
library(magrittr)

Using the GLPK callable library version 4.65


In [3]:
options(repr.matrix.max.rows = 600, repr.matrix.max.cols = 600)

#### Constantes y Funciones de Apoyo

In [4]:
dataset.root <- "./data/"
dataset.folder <- "./data/knapsack/"
dataset.folder.simple <- paste0(dataset.folder, "simple/")
dataset.folder.multi  <- paste0(dataset.folder, "multidimensional/")
dataset.folder.binpacking <- paste0(dataset.root, "binpacking/")

In [5]:
ReadSimpleKnapsack <- function(file.path) {
    con <- 
        file.path %>%
        file(open = "r")
    raw <- 
        con %>%
        readLines(warn = FALSE) %>% 
        strsplit("[[:blank:]]") %>%
        unlist() %>%
        as.double()
    close(con)
    list(
        n        = raw[1],
        capacity = raw[2] ,
        value    = raw[seq(4, length(raw), by = 3)],
        weight   = raw[seq(5, length(raw), by = 3)]
    ) %>%
    return()
}

In [6]:
SimpleKnapsackSubSet <- function(p, selected, capacity = p$capacity){
    if (sum(selected) > 0) {
       list(n        = sum(selected),
            capacity = capacity,
            value    = p$value[selected],
            weight   = p$weight[selected]
        ) %>%
        return()
    }
}

In [7]:
ReadMultidimensionalKnapsack <- function(file.path) {
    con <- 
        file.path %>%
        file(open = "r")
    raw <- 
        con %>%
        readLines(warn = FALSE) %>% 
        strsplit("[[:blank:]]") %>%
        unlist() %>%
        as.double()
    close(con)
    list(n        = raw[1],
         m        = raw[2],
         objval   = raw[3],
         value    = raw[4:(raw[1]+ 3)],
         weight   = matrix(raw[(4 + raw[1]):(3 + raw[1] * (1 + raw[2]))], 
                           ncol = raw[1], byrow = TRUE),
         capacity = raw[(4 + raw[1] * (1 + raw[2]) ):length(raw)]
    ) %>%
    return()
}

In [8]:
ReadBinPacking <- function(file.path) {
    con <- 
        file.path %>%
        file(open = "r")
    raw <- 
        con %>%
        readLines(warn = FALSE) %>% 
        strsplit("[[:blank:]]") %>%
        unlist()
    close(con)
    
    name <- raw[1]
    raw.numeric <- 
        raw[2:length(raw)] %>%
        as.double()

    list(name     = name,
         capacity = raw.numeric[1],
         n        = raw.numeric[2],
         objval   = raw.numeric[3],
         size     = raw.numeric[4:length(raw.numeric)]
    ) %>%
    return()
}

In [9]:
Solve <- function(f.obj, f.con, f.dir, f.rhs, ...) {
    Rglpk_solve_LP(f.obj, f.con, f.dir, f.rhs, ...)
}

## Problema de la Mochila

#### Descripción:
TODO

### Mochila Simple

#### Descripción:
TODO

#### Modelo:
TODO

In [10]:
KnapsackSimpleExact <- function(p, types = "B", max = TRUE, ...) {
    Solve(p$value, rbind(matrix(p$weight, nrow = 1, byrow = TRUE), diag(p$n)), 
          rep("<=", length(p$capacity) + p$n), c(p$capacity, rep(1, p$n)), types = types, 
          max = max, ...)
}

#### Heurísticas:
TODO

In [11]:
KnapsackSimpleRounding <- function(p, InitialSolution = KnapsackSimpleExact, 
                                   RoundingFunction = floor, ...) {
    sol.exact <- InitialSolution(p, types = "C", ...)
    
    sol <- list()
    sol$solution = RoundingFunction(sol.exact$solution)
    sol$objval   = sum(sol$solution * p$value)
    sol$slack    = p$capacity - sum(sol$solution * p$weight)
    return(sol)
}

In [12]:
RelativeCriterion <- function(p) {
    p$value/p$weight
}

In [13]:
KnapsackSimpleGreedy <- function(p, Criterion = RelativeCriterion, ...) {
    sol <- list(solution = rep(0, p$n), 
                objval = 0, slack = p$capacity)
    
    g <- Criterion(p)
    i <- which(g == max(g[sol$solution == 0]) & sol$solution == 0)[1]
    
    while (!is.na(i) && sol$slack - p$weight[i] > 0) {
        sol$solution[i] <- 1
        sol$objval <- sol$objval + p$value[i]
        sol$slack <- sol$slack - p$weight[i]
        
        if (any(sol$solution == 0)) {
            i <- which(g == max(g[sol$solution == 0]) & sol$solution == 0)[1]    
        } else {
            i <- NA
        }
    }
    return(sol)
}

In [14]:
KnapsackSimpleLocalSearchBest <- function(p, sol, idx0, idx1) {
    best = list(v = 0, i = 0, j = 0)
    for (i in idx0) {
        for (j in idx1) {
            if (sol$slack + p$weight[j] - p$weight[i] > 0) {
                v.temp <- p$value[i] - p$value[j]
                if(best$v < v.temp) {
                    best$v <- v.temp
                    best$i <- i
                    best$j <- j
                }   
            }
        }
    }
    return(best)
}

In [15]:
KnapsackSimpleLocalSearch <- function(p, InitialSolution = KnapsackSimpleGreedy, ...) {
    sol <- InitialSolution(p, ...)
    
    idx0 <- which(sol$solution == 0)
    idx1 <- which(sol$solution == 1)
    
    best <- KnapsackSimpleLocalSearchBest(p, sol, idx0, idx1)
    while (best$v > 0) {
        sol$solution[best$i] <- 1
        sol$solution[best$j] <- 0
        sol$slack <- sol$slack + p$weight[best$j] - p$weight[best$i]
        sol$objval <- sol$objval + best$v
        
        idx0 <- c(idx0[idx0 != best$i], best$j)
        idx1 <- c(idx1[idx1 != best$j], best$i)
        
        best <- KnapsackSimpleLocalSearchBest(p, sol, idx0, idx1)
    }
    return(sol)    
}

### Ejemplos

In [16]:
GenerateSimpleKnapsackSolutions <- function(p, ...) {
    sol <- list()
    
    s <- KnapsackSimpleExact(p, ...)
    sol$exact <- list(solution = matrix(s$solution, ncol = p$n, byrow = TRUE),
                      objval   = s$optimum,
                      slack    = p$capacity - s$auxiliary$primal[1])

    s <- KnapsackSimpleExact(p, types = "C", ...)
    sol$relaxed <- list(solution = matrix(s$solution, ncol = p$n, byrow = TRUE),
                        objval   = s$optimum,
                        slack    = p$capacity - s$auxiliary$primal[1])

    s <- KnapsackSimpleRounding(p, ...)
    sol$rounded <- list(solution = matrix(s$solution, ncol = p$n, byrow = TRUE),
                        objval   = s$objval,
                        slack    = s$slack)

    s <- KnapsackSimpleGreedy(p, ...)
    sol$greedy <- list(solution = matrix(s$solution, ncol = p$n, byrow = TRUE),
                       objval   = s$objval,
                       slack    = s$slack)

    s <- KnapsackSimpleLocalSearch(p, ...)
    sol$local <- list(solution = matrix(s$solution, ncol = p$n, byrow = TRUE),
                      objval   = s$objval, 
                      slack    = s$slack)
    
    return(sol)
}

##### gen100.1

In [17]:
gen100.1 <- ReadSimpleKnapsack(paste0(dataset.folder.simple, "gen100_1.txt"))

In [18]:
GenerateSimpleKnapsackSolutions(gen100.1)

1,0,1,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,1,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
1,0,1,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.5,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0
1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0
1,0,1,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0
1,0,1,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0


TODO(@garciparedes): Analyze results

##### gen100.2

In [19]:
gen100.2 <- ReadSimpleKnapsack(paste0(dataset.folder.simple, "gen100_2.txt"))

In [20]:
GenerateSimpleKnapsackSolutions(gen100.2)

0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0
0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0.4210526,0,0,1,0,0,0
0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0
0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0


TODO(@garciparedes): Analyze results

##### gen100.3

In [21]:
gen100.3 <- ReadSimpleKnapsack(paste0(dataset.folder.simple, "gen100_3.txt"))

In [22]:
GenerateSimpleKnapsackSolutions(gen100.3)

0,0,0,0,0,0,1,0,0,1,0,1,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,1
1,0,0,0,0,0,1,0,0,1,0,1,0.07692308,1,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1
1,0,0,0,0,0,1,0,0,1,0,1,0,1,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
1,0,0,0,0,0,1,0,0,1,0,1,0,1,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1
1,0,0,0,0,0,1,0,0,1,0,1,0,1,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1


TODO(@garciparedes): Analyze results

##### gen100.4

In [23]:
gen100.4 <- ReadSimpleKnapsack(paste0(dataset.folder.simple, "gen100_4.txt"))

In [24]:
GenerateSimpleKnapsackSolutions(gen100.4)

0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,1,1,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0
0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0.7333333,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,1,1,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0
0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,1,1,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0
0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,1,1,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0


TODO(@garciparedes): Analyze results

##### gen100.5

In [25]:
gen100.5 <- ReadSimpleKnapsack(paste0(dataset.folder.simple, "gen100_5.txt"))

In [26]:
GenerateSimpleKnapsackSolutions(gen100.5)

0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0.3846154,0,1,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0


TODO(@garciparedes): Analyze results

##### gen200.1

In [27]:
gen200.1 <- ReadSimpleKnapsack(paste0(dataset.folder.simple, "gen200_1.txt"))

In [28]:
GenerateSimpleKnapsackSolutions(gen200.1)

0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,1,1,0,0,0,1,0,0,1,0,1,1,1,0,0,0,1,0,1,1,0,1,1,0,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,1,0,0,1,1,1,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,1,1,0,1,1,1,0,0,1,0,0,0,0,0,1,1,1,0,0,0,1,0,0,1,0,0,0,0,0,1
0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0.08333333,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,1,1,0,0,0,1,0,0,1,0,1,1,1,0,0,0,1,0,1,1,0,1,1,0,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,1,0,0,1,1,1,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,1,1,1,1,1,1,0,0,1,0,0,0,0,0,1,1,1,0,0,0,1,0,0,1,0,0,0,0,0,1
0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,1,1,0,0,0,1,0,0,1,0,1,1,1,0,0,0,0,0,1,1,0,1,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,1,0,0,1,1,1,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,1,1,0,1,1,1,0,0,1,0,0,0,0,0,1,1,1,0,0,0,1,0,0,1,0,0,0,0,0,1
0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,1,1,0,0,0,1,0,0,1,0,1,1,1,0,0,0,1,0,1,1,0,1,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,1,0,0,1,1,1,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,1,1,1,1,1,1,0,0,1,0,0,0,0,0,1,1,1,0,0,0,1,0,0,1,0,0,0,0,0,1
0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,1,1,0,0,0,1,0,0,1,0,1,1,1,0,0,0,1,0,1,1,0,1,1,0,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,1,0,0,1,1,1,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,1,1,1,1,1,1,0,0,1,0,0,0,0,0,1,1,1,0,0,0,1,0,0,1,0,0,0,0,0,1


##### gen200.2

In [29]:
gen200.2 <- ReadSimpleKnapsack(paste0(dataset.folder.simple, "gen200_2.txt"))

In [30]:
GenerateSimpleKnapsackSolutions(gen200.2)

0,0,0,0,1,1,1,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,1,0,1,1,0,0,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,1,0,1,0,1,0,1,0,0,0,1,1,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,1,1,0,0,1,0,1,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,1,0,0,1,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,1,0,0
0,0,0,0,1,1,1,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,1,0,0,0,0,1,0,1,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,1,0,1,0,1,0,1,0,0,0,1,1,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,1,1,0,0,1,0,1,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,1,0,0,1,1,1,0,0,1,0.5,1,0,0,0,0,0,0,0,0,1,0,0
0,0,0,0,1,1,1,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,1,0,1,0,1,0,1,0,0,0,1,1,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0
0,0,0,0,1,1,1,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,1,0,0,0,0,1,0,1,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,1,0,1,0,1,0,1,0,0,0,1,1,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,1,1,0,0,1,0,1,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,1,0,0,1,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0
0,0,0,0,1,1,1,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,1,0,1,0,1,0,1,0,0,0,1,1,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,1,1,0,0,1,0,1,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,1,0,0,1,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,1,0,0


TODO(@garciparedes): Analyze results

##### gen200.3

In [31]:
gen200.3 <- ReadSimpleKnapsack(paste0(dataset.folder.simple, "gen200_3.txt"))

In [32]:
GenerateSimpleKnapsackSolutions(gen200.3)

1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,1,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,1,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,1,1,0,1,1,0,0,0,1,1,0,0,0,0,1,1,1,0,0,0,0,0
1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,1,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0.1333333,1,0,1,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,1,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,1,1,0,1,1,0,0,0,1,1,0,0,0,0,1,1,1,0,0,0,0,0
1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,1,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,1,1,0,1,1,0,0,0,1,1,0,0,0,0,1,1,1,0,0,0,0,0
1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,1,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,1,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,1,1,0,1,1,0,0,0,1,1,0,0,0,0,1,1,1,0,0,0,0,0
1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,1,1,0,0,1,0,1,0,1,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,1,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,1,1,0,1,1,0,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0


TODO(@garciparedes): Analyze results

##### gen200.4

In [33]:
gen200.4 <- ReadSimpleKnapsack(paste0(dataset.folder.simple, "gen200_4.txt"))

In [34]:
GenerateSimpleKnapsackSolutions(gen200.4)

0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,0,1,0,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,1,1,0,1,1,0,1,1,0,1,0,1,0,1,0,1,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,1,1,0,0,0,0,0,1,1,0,0,1,1,0,1,0,0,0,0,0,0,1,1,0,1,1,1,1,0,0,0,0,0,0
0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,0,1,0,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0.7368421,0,1,1,0,1,1,0,1,0,1,0,1,0,1,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,1,0,0,1,1,0,1,0,0,0,0,0,0,1,1,0,1,1,1,1,0,0,0,0,0,0
0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,1,0,1,1,0,1,0,1,0,1,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,1,0,0,1,1,0,1,0,0,0,0,0,0,1,1,0,1,1,1,1,0,0,0,0,0,0
0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,0,1,0,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,1,1,0,1,1,0,1,1,0,1,0,1,0,1,0,1,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,1,0,0,1,1,0,1,0,0,0,0,0,0,1,1,0,1,1,1,1,0,0,0,0,0,0
0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,0,0,0,1,1,1,0,0,0,0,1,0,1,0,0,1,0,0,1,1,0,1,1,0,1,1,0,1,0,1,0,1,0,1,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,1,0,0,1,1,0,1,0,0,0,0,0,0,1,1,0,1,1,1,1,0,0,0,0,0,0


TODO(@garciparedes): Analyze results

##### gen200.5

In [35]:
gen200.5 <- ReadSimpleKnapsack(paste0(dataset.folder.simple, "gen200_5.txt"))

In [36]:
GenerateSimpleKnapsackSolutions(gen200.5)

0,1,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,1,1,0,0,1,0,0,0,1,0,1,0,0,0,1,1,0,1,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,1,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,1,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,1
0,1,0,0,1,0,0,0.6666667,1,0,0,1,0,0,1,0,0,1,1,1,0,0,1,0,0,0,1,0,1,0,0,0,1,1,0,1,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,1,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,1
0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,1,1,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,1,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,1,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0
0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,1,1,0,0,1,0,0,0,1,0,1,0,0,0,1,1,0,1,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,1,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,1
0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,1,1,0,0,1,0,0,0,1,0,1,0,0,0,1,1,0,1,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,1,0,0,1,1,1,1,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,1,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,1


TODO(@garciparedes): Analyze results

##### gen300.1

In [37]:
gen300.1 <- ReadSimpleKnapsack(paste0(dataset.folder.simple, "gen300_1.txt"))

In [38]:
GenerateSimpleKnapsackSolutions(gen300.1)

1,1,0,0,1,1,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0,0,1,1,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,1,1,0,1,1,1,0,0,1,0,0,1,0,1,1,1,1,1,1,0,0,1,0,1,1,0,1,0,0,1,0,1,1,1,0,1,1,0,0,1,1,0,0,1,0,0,0,1,1,1,0,0,0,0,1,1,1,1,0,1,1,0,1,1,0,1,0,0,0,1,1,0,1,1,1,1,1,0,1,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,1,1,0,1,1,1,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,1,1,1,0,1,1,1,1,1,1,0,1,0,1,1,1,1,1,0,0,1,0,1,1,0,0,0,1,0,1,0,0,0,0,0,1,1,0,1,1,0,1,1,1,1,0,0,0,0,1,1,0,1,0,1,1,1,0,0,1,1,1,0,1,0,0,0,1,0,1,1,1,1,1,1,1,1,0,1,1,1,0,1,0,1,0,1,0,0,1,0,0,1,0,1,1,0,0,1,0,0,1,0,1,0,0,0,1,1,0,1,0
1,1,0,0,1,1,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0,0,1,1,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,1,1,0,1,1,1,0,0,1,0,0,1,0,1,1,1,1,1,1,0,0,1,0,1,1,0,1,0,0,1,0,1,1,1,0,1,1,0,0,1,1,0,0,1,0,0,0,1,1,1,0,0,0,0,1,1,1,1,0,1,1,0,1,1,0,1,0,0,0,1,1,0,1,1,1,1,1,0,1,0,1,0,1,0,0,0,0,1,0,0,1,0,7.565377e-15,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,1,1,0,1,1,1,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,1,1,1,0,1,1,1,1,1,1,0,1,0,1,1,1,1,1,0,0,1,0,1,1,0,0,0,1,0,1,0,0,0,0,0,1,1,0,1,1,0,1,1,1,1,0,0,0,0,1,1,0,1,0,1,1,1,0,0,1,1,1,0,1,0,0,0,1,0,1,1,1,1,1,1,1,1,0,1,1,1,0,1,0,1,0,1,0,0,1,0,0,1,0,1,1,0,0,1,0,0,1,0,1,0,0,0,1,1,0,1,0
1,1,0,0,1,1,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0,0,1,1,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,1,1,0,1,1,1,0,0,1,0,0,1,0,1,1,1,1,1,1,0,0,1,0,1,1,0,1,0,0,1,0,1,1,1,0,1,1,0,0,1,1,0,0,1,0,0,0,1,1,1,0,0,0,0,1,1,1,1,0,1,1,0,0,1,0,1,0,0,0,1,1,0,1,1,1,1,1,0,1,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,1,1,0,1,1,1,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,1,1,1,0,1,1,1,1,1,1,0,1,0,1,1,1,1,1,0,0,1,0,1,1,0,0,0,1,0,1,0,0,0,0,0,1,1,0,1,1,0,1,1,1,1,0,0,0,0,1,1,0,1,0,1,1,1,0,0,1,1,1,0,1,0,0,0,1,0,1,1,1,1,1,1,1,1,0,1,1,1,0,1,0,1,0,1,0,0,1,0,0,1,0,1,1,0,0,1,0,0,1,0,1,0,0,0,1,1,0,1,0
1,1,0,0,1,1,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0,0,1,1,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,1,1,0,1,1,0,0,0,1,0,0,1,0,1,1,1,1,1,1,0,0,1,0,1,1,0,1,0,0,1,0,1,1,1,0,1,1,0,0,1,1,0,0,1,0,0,0,1,1,1,0,0,0,0,1,1,1,1,0,1,1,0,1,1,0,1,0,0,0,1,1,0,1,1,1,1,1,0,1,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,1,1,0,1,1,1,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,1,1,1,0,1,1,1,1,1,1,0,1,0,1,1,1,1,1,0,0,1,0,1,1,0,0,0,1,0,1,0,0,0,0,0,1,1,0,1,1,0,1,1,1,1,0,0,0,0,1,1,0,1,0,1,1,1,0,0,1,1,1,0,1,0,0,0,1,0,1,1,1,1,1,1,1,1,0,1,1,1,0,1,0,1,0,1,0,0,1,0,0,1,0,1,1,0,0,1,0,0,1,0,1,0,0,0,1,1,0,1,0
1,1,0,0,1,1,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0,0,1,1,0,0,1,0,0,1,0,1,1,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,1,1,0,1,1,1,0,0,1,0,0,1,0,0,1,1,1,1,1,0,0,1,0,1,0,0,1,0,0,1,0,1,1,1,0,1,1,0,0,1,1,0,0,1,0,0,0,1,1,1,0,0,0,0,1,1,1,1,0,1,1,0,0,1,0,1,0,0,0,1,1,0,1,1,1,1,1,0,1,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,1,1,0,1,1,1,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,1,1,1,0,1,1,1,1,1,1,0,1,0,1,1,1,1,1,0,0,1,0,1,1,0,0,0,1,0,1,0,0,0,0,0,1,1,0,1,1,0,1,1,1,1,0,0,0,0,1,1,0,1,0,1,1,1,0,0,1,1,1,0,1,0,0,0,1,0,1,1,1,1,1,1,1,1,0,1,1,1,0,1,0,1,0,1,0,1,1,0,0,1,0,1,1,0,0,1,0,0,1,0,1,0,0,0,1,1,0,1,0


TODO(@garciparedes): Analyze results

##### gen300.2

In [39]:
gen300.2 <- ReadSimpleKnapsack(paste0(dataset.folder.simple, "gen300_2.txt"))

In [40]:
GenerateSimpleKnapsackSolutions(gen300.2)

0,0,0,1,0,1,0,1,1,0,0,1,1,1,0,0,0,1,1,0,0,0,0,1,0,1,0,0,1,1,1,0,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,1,0,0,1,0,0,0,1,1,1,0,1,1,0,1,0,0,1,0,1,1,1,1,0,1,0,1,0,1,0,1,0,1,1,0,0,1,1,0,0,0,1,1,0,1,1,1,1,1,0,1,1,1,0,0,1,1,1,1,0,0,0,1,1,0,1,0,0,0,0,1,0,0,1,0,1,0,1,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,1,1,0,1,1,1,0,0,0,0,1,1,1,0,0,0,0,1,1,0,0,0,0,1,1,1,1,1,0,1,0,1,0,1,1,0,1,1,0,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,0,0,1,1,1,0,0,0,0,1,0,1,1,1,1,1,1,1,0,1,0,1,0,1,0,0,1,0,1,1,0,1,0,0,0,0,1,1,1,0,1,1,1,1,1,0,0,1,1,1,0,0,0,1,0,1,0,1,0,1,1,1,1,0,0,1,1,0,0,1,1,0,1,1,0,0,1,0,1,0,0,0,0,0,1,1,0,1,0,0,1,1,0,1,0,1
0,0,0,1,0,1,0,1,1,0,0,1,1,1,0,0,0,1,1,0,0,1,0,1,0,1,0,0,1,1,1,0,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,1,0,0,1,0,0,0,1,1,1,0,1,1,0,1,0,0,1,0,1,1,1,1,0,1,0,1,0,1,0,1,0,1,1,0,0,1,1,0,0,0,1,1,0,1,1,1,1,1,0,1,1,1,0,0,1,1,1,1,0,0,0,1,1,0,1,0,0,0,0,1,0,0,1,0,1,0,1,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0.1111111,1,0,1,1,0,1,1,1,0,0,0,0,1,1,1,0,0,0,0,1,1,0,0,0,0,1,1,1,1,1,0,1,0,1,0,1,1,0,1,1,0,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,0,0,1,1,1,0,0,0,0,1,0,1,1,1,1,1,1,1,0,1,0,1,0,1,0,0,1,0,1,1,0,1,0,0,0,0,1,1,1,0,1,1,1,1,1,0,0,1,1,1,0,0,0,1,0,1,0,0,0,1,1,1,1,0,0,1,1,0,0,1,1,0,1,1,0,0,1,0,1,0,0,0,0,0,1,1,0,1,0,0,1,1,0,1,0,1
0,0,0,1,0,1,0,1,1,0,0,1,1,1,0,0,0,1,1,0,0,1,0,0,0,1,0,0,1,1,1,0,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,1,0,0,1,0,0,0,1,1,1,0,1,1,0,0,0,0,1,0,1,1,1,1,0,1,0,1,0,1,0,1,0,1,1,0,0,1,1,0,0,0,1,1,0,1,1,1,0,1,0,1,1,1,0,0,1,1,1,1,0,0,0,1,1,0,1,0,0,0,0,1,0,0,1,0,1,0,1,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,0,1,0,1,1,1,0,0,0,0,1,1,1,0,0,0,0,1,1,0,0,0,0,1,1,1,1,1,0,1,0,0,0,1,1,0,1,1,0,1,0,1,0,1,1,1,1,1,0,1,0,1,1,0,1,1,0,0,1,1,1,0,0,0,0,1,0,1,1,1,1,1,1,1,0,1,0,1,0,1,0,0,1,0,1,1,0,1,0,0,0,0,1,1,1,0,1,1,1,1,1,0,0,1,1,1,0,0,0,1,0,1,0,0,0,1,1,1,1,0,0,1,1,0,0,1,1,0,1,1,0,0,1,0,1,0,0,0,0,0,1,1,0,1,0,0,1,1,0,1,0,1
0,0,0,1,0,1,0,1,1,0,0,1,1,1,0,0,0,1,1,0,0,1,0,1,0,1,0,0,1,1,1,0,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,1,0,0,1,0,0,0,1,1,1,0,1,1,0,1,0,0,1,0,1,1,1,1,0,1,0,1,0,1,0,1,0,1,1,0,0,1,1,0,0,0,1,1,0,1,1,1,1,1,0,1,1,1,0,0,1,1,1,1,0,0,0,1,1,0,1,0,0,0,0,1,0,0,1,0,1,0,1,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,1,1,0,1,1,1,0,0,0,0,1,1,1,0,0,0,0,1,1,0,0,0,0,1,1,1,1,1,0,1,0,1,0,1,1,0,1,1,0,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,0,0,1,1,1,0,0,0,0,1,0,1,1,1,1,1,1,1,0,1,0,1,0,1,0,0,1,0,1,1,0,1,0,0,0,0,1,1,1,0,1,1,1,1,1,0,0,1,1,1,0,0,0,1,0,1,0,0,0,1,1,1,1,0,0,1,1,0,0,1,1,0,1,1,0,0,1,0,1,0,0,0,0,0,1,1,0,1,0,0,1,1,0,1,0,1
0,0,0,1,0,1,0,1,1,0,0,1,1,1,0,0,0,1,1,0,0,1,0,1,0,1,0,0,1,1,1,0,1,1,1,0,0,0,0,0,0,0,1,1,1,1,1,1,0,0,1,0,0,0,1,1,1,0,1,1,0,1,0,0,1,0,1,1,1,1,0,1,0,1,0,1,0,1,0,1,1,0,0,1,1,0,0,0,1,1,0,1,1,1,1,1,0,1,1,1,0,0,1,1,1,1,0,0,0,1,1,0,1,0,0,0,0,1,0,0,1,0,1,0,1,1,0,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,0,1,0,1,1,1,0,0,0,0,1,1,1,0,0,0,0,1,1,0,0,0,0,1,1,1,1,1,0,1,0,1,0,1,1,0,1,1,0,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,0,1,1,1,0,0,0,0,1,0,1,1,1,1,1,1,1,0,1,0,1,0,1,0,0,1,0,1,1,0,1,0,0,0,0,1,1,1,0,1,1,1,1,1,0,0,1,1,1,0,0,0,1,0,1,0,0,0,1,1,1,1,0,0,1,1,0,0,1,1,0,1,1,0,0,1,0,1,0,0,0,0,0,1,1,0,1,0,0,1,1,0,1,0,1


TODO(@garciparedes): Analyze results

##### gen300.3

In [41]:
gen300.3 <- ReadSimpleKnapsack(paste0(dataset.folder.simple, "gen300_3.txt"))

In [42]:
GenerateSimpleKnapsackSolutions(gen300.3)

0,0,1,0,1,0,0,1,1,1,0,1,1,0,1,0,0,1,1,1,1,0,1,1,1,0,1,0,1,0,1,0,0,0,1,0,1,0,1,1,0,0,1,0,1,1,1,1,0,0,0,0,1,1,1,1,1,0,0,1,0,0,1,0,1,0,1,1,0,0,0,0,0,1,1,1,0,0,0,1,1,1,0,1,1,0,0,0,0,1,0,1,0,1,1,0,1,0,1,1,1,1,0,1,1,0,0,1,1,1,0,1,0,1,1,0,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,1,0,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,0,1,1,0,0,1,0,1,0,0,1,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,1,1,1,1,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,1,1,1,0,0,1,1,0,1,1,1,1,1,1,0,0,1,1,1,0,1,1,0,1,0,1,0,1,1,0,0,0,1,1,1,1,1,1,0,0,1,0,0,0,1,1,1,1,1,0,1,1,0,0,1,1,0,1,1,1,0,1,0,0,0,1,1
0,0,1,0,1,0,0,1,1,1,0,1,1,0,1,0,0,1,1,1,1,0,1,1,1,0,1,0,1,0,1,0,0,0,1,0,1,0,1,1,0,0,1,0,1,1,1,1,0,0,0,0,1,1,1,1,1,0,0,1,0,0,0,0,1,0,1,1,0,0,0,0,0,1,1,1,0,0,0,1,1,1,0,1,1,0,0,0,0,1,0,1,0,1,1,0,1,0,1,1,1,1,0,1,1,0,0,1,1,1,0,1,0,1,1,0,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,1,0,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,0,1,1,0,0,1,0,1,0,0,1,0,1,0,1,1,0,1,0,0.7368421,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,1,1,1,1,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,1,1,1,0,0,1,1,0,1,1,1,1,1,1,0,0,1,1,1,0,1,1,0,1,0,1,0,1,1,0,0,0,1,1,1,1,1,1,0,0,1,0,0,0,1,1,1,1,1,0,1,1,0,0,1,1,0,1,1,1,0,1,0,0,0,1,1
0,0,1,0,0,0,0,1,1,1,0,1,1,0,1,0,0,1,1,1,1,0,1,1,1,0,1,0,1,0,1,0,0,0,1,0,1,0,1,1,0,0,1,0,1,1,1,0,0,0,0,0,1,0,1,1,1,0,0,1,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,1,0,0,0,1,1,1,0,0,1,0,0,0,0,1,0,1,0,1,0,0,1,0,1,1,1,1,0,1,1,0,0,0,1,1,0,1,0,1,1,0,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,1,0,1,1,1,1,1,0,0,0,0,0,0,0,1,0,1,1,0,1,1,0,0,1,0,1,0,0,1,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,1,1,1,1,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,1,1,1,0,0,1,1,0,1,1,1,1,1,1,0,0,1,1,1,0,1,1,0,1,0,1,0,1,1,0,0,0,1,1,1,1,1,1,0,0,1,0,0,0,1,1,1,1,1,0,1,1,0,0,1,0,0,1,1,1,0,1,0,0,0,1,1
0,0,1,0,1,0,0,1,1,1,0,1,1,0,1,0,0,1,1,1,1,0,1,1,1,0,1,0,1,0,1,0,0,0,1,0,1,0,1,1,0,0,1,0,1,1,1,1,0,0,0,0,1,1,1,1,1,0,0,1,0,0,0,0,1,0,1,1,0,0,0,0,0,1,1,1,0,0,0,1,1,1,0,1,1,0,0,0,0,1,0,1,0,1,1,0,1,0,1,1,1,1,0,1,1,0,0,1,1,1,0,1,0,1,1,0,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,1,0,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,0,1,1,0,0,1,0,1,0,0,1,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,1,1,1,1,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,1,1,1,0,0,1,1,0,1,1,1,1,1,1,0,0,1,1,1,0,1,1,0,1,0,1,0,1,1,0,0,0,1,1,1,1,1,1,0,0,1,0,0,0,1,1,1,1,1,0,1,1,0,0,1,1,0,1,1,1,0,1,0,0,0,1,1
0,0,1,0,1,0,0,1,1,1,0,1,1,0,1,0,0,1,1,1,1,0,1,1,1,0,1,0,1,0,1,0,0,0,1,0,1,0,1,1,0,0,1,0,1,0,1,1,0,0,0,0,1,1,1,1,1,0,0,1,0,0,0,0,1,0,1,1,0,0,0,0,0,1,1,1,0,0,0,1,1,1,0,1,1,0,0,0,0,1,0,1,0,1,1,0,1,0,1,1,1,1,0,1,1,0,0,1,1,1,0,1,0,1,1,0,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,1,0,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,0,1,1,0,0,0,0,1,0,0,1,0,1,0,1,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,1,1,1,1,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,1,1,1,0,0,1,1,0,1,1,1,1,1,1,0,0,1,1,1,0,1,1,0,1,0,1,0,1,1,0,0,0,1,1,1,1,1,1,0,0,1,0,0,0,1,1,1,1,1,0,1,1,0,0,1,1,0,1,1,1,0,1,0,0,0,1,1


TODO(@garciparedes): Analyze results

##### gen300.4

In [43]:
gen300.4 <- ReadSimpleKnapsack(paste0(dataset.folder.simple, "gen300_4.txt"))

In [44]:
GenerateSimpleKnapsackSolutions(gen300.4)

0,0,1,1,0,0,0,0,0,1,1,0,1,0,1,1,1,1,0,1,0,0,1,1,1,0,0,0,0,1,0,1,1,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,1,1,0,1,0,0,0,1,0,1,0,1,0,1,0,1,1,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,1,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,1,0,0,0,1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,0,1,0,0,0,1,1,1,1,0,0,1,0,1,0,1,0,0,0,1,1,1,1,0,0,0,0,1,1,0,0,1,0,1,0,0,1,1,1,0,1,1,0,1,1,1,1,0,1,1,0,1,1,0,0,0,0,1,1,0,1,1,1,0,0,0,1,1,1,0,1,1,0,0,1,0,1,1,1,1,0,1,0,1,0,0,0,1,1,1,0,1,0,1,1,1,0,0,0,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,1,0,1,1,1,1,0,0,1,1,1,1,0,1,1,1,0,0,1,1,0,1,1,1,1,0,1,0,1,1,0,1,0,1,1,0,0,0,1,0,1,1,1,0,1,0,0,1
0,0,1,1,0,0,0,0,0,1,0,0,1,0,1,1,1,1,0,1,0,0,1,1,1,0,0,0,0,1,0,1,1,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,1,1,0,1,0,0,0,1,0,1,0,1,0,1,0,1,1,1,1,0,1,0,0,1,1,0,0,0,0,0,0,0,1,1,0,1,1,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,1,0,0,0,1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,0,1,0,0,0,1,1,1,1,0,0,1,0,1,0,1,0,0,0,1,1,1,1,0,0,0,0,1,1,0,0,1,0,1,0,0,1,1,1,0,1,1,0,1,1,1,1,0,1,1,0,1,1,0,0,0,0,1,1,0,1,1,1,0,0,0,1,1,1,0,1,1,0,0,1,0,1,1,1,1,0,1,0,1,0,0,0,1,1,1,0,1,0,1,1,1,0,0,0,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,0.9411765,0,1,1,1,1,0,0,1,1,1,1,0,1,1,1,0,0,1,1,0,1,1,1,1,0,1,0,1,1,0,1,0,1,1,0,0,0,1,0,1,1,1,0,1,0,0,1
0,0,1,1,0,0,0,0,0,1,0,0,1,0,1,1,1,1,0,1,0,0,1,1,1,0,0,0,0,1,0,1,1,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0,1,0,1,0,1,1,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,1,0,0,0,1,1,1,1,1,0,0,0,0,0,1,1,0,1,1,1,0,0,0,0,1,1,1,1,0,1,0,0,0,1,1,1,1,0,0,1,0,1,0,1,0,0,0,1,1,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,1,0,1,1,0,1,1,1,1,0,1,1,0,1,1,0,0,0,0,1,1,0,1,1,1,0,0,0,1,1,1,0,1,1,0,0,1,0,1,1,1,1,0,1,0,1,0,0,0,1,1,1,0,1,0,1,1,1,0,0,0,1,1,1,1,1,1,1,1,0,1,1,1,1,0,0,0,0,1,1,1,1,0,0,1,1,1,1,0,1,1,1,0,0,1,1,0,1,1,1,1,0,1,0,1,1,0,1,0,1,1,0,0,0,1,0,1,1,1,0,1,0,0,1
0,0,1,1,0,0,0,0,0,1,0,0,1,0,1,1,1,1,0,1,0,0,1,1,1,0,0,0,0,1,0,1,1,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,1,1,0,1,0,0,0,1,0,1,0,1,0,1,0,1,1,1,1,0,1,0,0,1,1,0,0,0,0,0,0,0,1,1,0,1,1,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,1,0,0,0,1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,0,1,0,0,0,1,1,1,1,0,0,1,0,1,0,1,0,0,0,1,1,1,1,0,0,0,0,1,1,0,0,1,0,1,0,0,1,1,1,0,1,1,0,1,1,1,1,0,1,1,0,1,1,0,0,0,0,1,1,0,1,1,1,0,0,0,1,1,1,0,1,1,0,0,1,0,1,1,1,1,0,1,0,1,0,0,0,1,1,1,0,1,0,1,1,1,0,0,0,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,0,0,1,1,1,1,0,0,1,1,1,1,0,1,1,1,0,0,1,1,0,1,1,1,1,0,1,0,1,1,0,1,0,1,1,0,0,0,1,0,1,1,1,0,1,0,0,1
0,0,1,1,0,0,0,0,0,1,0,0,1,0,1,1,1,1,0,1,0,0,1,1,1,0,0,0,0,1,0,1,1,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,1,1,0,1,0,0,0,1,0,1,0,1,0,1,0,1,1,1,1,0,1,0,0,1,1,0,0,0,0,0,0,0,1,0,0,1,1,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,1,0,0,0,1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,0,1,0,0,0,1,1,1,1,0,0,1,0,1,0,1,0,0,0,1,1,1,1,0,0,0,0,1,1,0,0,1,0,1,0,0,1,1,1,0,1,1,0,1,1,1,1,0,1,1,0,1,0,0,0,0,0,1,1,0,1,1,1,0,0,0,1,1,1,0,1,1,0,0,1,0,1,1,1,1,0,1,0,1,0,0,0,1,1,1,0,1,0,1,1,1,0,0,0,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,1,0,1,1,1,1,0,0,1,1,1,1,0,1,1,1,0,0,1,1,0,1,1,1,1,0,1,0,1,1,0,1,0,1,1,1,0,0,1,0,1,1,1,0,1,0,0,1


TODO(@garciparedes): Analyze results

##### gen300.5

In [45]:
gen300.5 <- ReadSimpleKnapsack(paste0(dataset.folder.simple, "gen300_5.txt"))

In [46]:
GenerateSimpleKnapsackSolutions(gen300.5)

1,0,1,1,0,0,0,1,1,1,1,1,0,1,0,1,1,0,1,1,0,0,0,1,1,0,1,1,0,0,1,0,1,0,1,0,0,0,1,0,1,1,1,0,0,1,0,0,0,1,1,0,1,1,0,1,1,1,1,0,0,0,1,1,1,1,0,1,1,1,1,1,0,1,0,0,1,0,1,1,1,1,0,1,1,1,0,0,0,1,1,1,0,0,0,1,0,0,1,1,0,1,1,1,0,1,1,0,1,0,1,1,1,1,0,0,0,1,1,0,0,1,0,0,1,0,0,1,1,1,0,0,0,1,0,1,1,1,1,1,0,1,1,0,1,1,0,0,0,1,1,0,0,0,1,0,1,1,0,0,1,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,1,1,0,0,1,1,1,1,0,1,1,1,1,0,1,1,0,1,1,1,1,0,0,0,1,0,1,1,0,1,0,0,1,0,1,1,0,0,0,1,1,1,1,0,0,0,1,0,0,0,1,0,0,0,1,1,0,1,1,0,1,1,0,1,1,1,1,0,1,0,0,0,1,0,0,0,1,1,1,1,1,1,0,0,0,1,1,1,0,0,0,0,1,1,1,1,1,1,0,0,0,1,0,1,0,0,1,0,1,0,0,1,1
1,0,1,1,0,0,0,1,1,1,1,1,0,1,0,1,1,0,1,1,0,0,0,1,1,0,1,1,0,0,1,0,1,0,1,0,0,0,1,0,1,1,1,0,0,1,0,0,0,1,1,0,1,1,0,1,1,1,1,0,0,0,1,1,1,1,0,1,1,1,1,1,0,1,0,0,1,0,1,1,1,1,0,1,1,1,0,0,0,1,1,1,0,0,0,1,0,0,1,1,0,1,1,1,0,1,1,0,1,0,1,1,1,1,0,0,0,1,1,0,0,1,0,0,1,0,0,1,1,1,0,0,0,1,0,1,1,1,1,1,0,1,1,0,1,1,0,0,0,1,1,0,0,0,1,0,1,1,0,0,1,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,1,1,0,0,1,1,1,1,0,1,1,1,1,0,1,1,0,1,1,1,1,0,0,0,1,0.7777778,1,1,0,1,0,0,1,0,1,1,0,0,0,1,1,1,1,0,0,0,1,0,0,0,1,0,0,0,1,1,0,1,1,0,1,1,0,1,1,1,1,0,1,0,0,0,1,0,0,0,1,1,1,1,1,1,0,0,0,1,1,1,0,0,0,0,1,0,1,1,1,1,0,0,0,1,0,1,0,0,1,0,1,0,0,1,1
1,0,1,1,0,0,0,1,1,1,1,1,0,1,0,1,1,0,1,1,0,0,0,1,1,0,1,1,0,0,1,0,1,0,1,0,0,0,1,0,1,1,1,0,0,1,0,0,0,1,1,0,0,1,0,1,1,1,1,0,0,0,1,1,1,1,0,1,1,1,1,1,0,1,0,0,1,0,1,1,1,1,0,1,1,1,0,0,0,1,1,1,0,0,0,1,0,0,1,1,0,1,1,1,0,1,1,0,1,0,1,1,1,1,0,0,0,1,1,0,0,1,0,0,1,0,0,1,1,1,0,0,0,0,0,1,1,1,1,1,0,1,1,0,1,1,0,0,0,1,1,0,0,0,1,0,1,1,0,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,1,1,0,0,1,1,1,1,0,1,1,1,1,0,1,1,0,1,1,1,1,0,0,0,1,0,1,1,0,1,0,0,1,0,1,1,0,0,0,1,1,1,1,0,0,0,1,0,0,0,1,0,0,0,1,1,0,1,1,0,1,1,0,1,1,1,1,0,1,0,0,0,1,0,0,0,1,1,1,1,1,1,0,0,0,1,1,1,0,0,0,0,1,0,1,1,1,1,0,0,0,1,0,1,0,0,1,0,1,0,0,1,1
1,0,1,1,0,0,0,1,1,1,1,1,0,1,0,1,1,0,1,1,0,0,0,1,1,0,1,1,0,0,1,0,1,0,1,0,0,0,1,0,1,1,1,0,0,1,0,0,0,1,1,0,1,1,0,1,1,1,1,0,0,0,1,1,1,1,0,1,1,1,1,1,0,1,0,0,1,0,1,1,1,1,0,1,1,1,0,0,0,1,1,1,0,0,0,1,0,0,1,1,0,1,1,1,0,1,1,0,1,0,1,1,1,1,0,0,0,1,1,0,0,1,0,0,1,0,0,1,1,1,0,0,0,1,0,1,1,1,1,1,0,1,1,0,1,1,0,0,0,1,1,0,0,0,1,0,1,1,0,0,1,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,1,1,0,0,1,1,1,1,0,1,1,1,1,0,1,1,0,1,1,1,1,0,0,0,1,0,1,1,0,1,0,0,1,0,1,1,0,0,0,1,1,1,1,0,0,0,1,0,0,0,1,0,0,0,1,1,0,1,1,0,1,1,0,1,1,1,1,0,1,0,0,0,1,0,0,0,1,1,1,1,1,1,0,0,0,1,1,1,0,0,0,0,1,0,1,1,1,1,0,0,0,1,0,1,0,0,1,0,1,0,0,1,1
1,0,1,1,0,0,0,1,1,1,1,1,0,1,0,1,1,0,1,1,0,0,0,1,1,0,1,1,0,0,1,1,1,0,1,0,0,0,1,0,1,1,1,0,0,1,0,0,0,1,1,0,1,1,0,1,1,1,1,0,0,0,1,1,1,1,0,1,1,1,1,1,0,1,0,0,1,0,1,1,1,1,0,1,1,0,0,0,0,1,1,1,0,0,0,1,0,0,1,1,0,1,1,1,0,1,1,0,1,0,1,1,1,1,0,0,0,1,1,0,0,1,0,0,1,0,0,1,1,1,0,0,0,1,0,1,1,1,1,1,0,1,1,0,1,1,0,0,0,1,1,0,0,0,1,0,1,1,0,0,1,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,1,1,0,0,1,1,1,1,0,1,1,1,1,0,1,1,0,1,1,1,1,0,0,0,1,1,1,1,0,1,0,0,1,0,1,1,0,0,0,1,1,1,1,0,0,0,1,0,0,0,1,0,0,0,1,1,0,1,1,0,1,1,0,1,1,1,1,0,1,0,0,0,1,0,0,0,1,1,1,1,1,1,0,0,0,1,1,1,0,0,0,0,1,0,1,1,1,0,0,0,0,1,0,1,0,0,1,0,1,0,0,1,1


TODO(@garciparedes): Analyze results

### Mochila Múltiple

#### Descripción:
TODO

#### Modelo:
TODO

In [47]:
KnapsackMultipleExact <- function(p, m, types = "B", max = TRUE, ...) {
    conditions <- matrix(rep(0,(m + p$n) * p$n * m), 
                         nrow = m + p$n, ncol = p$n * m, byrow = TRUE)
    
    for(i in 1:m) {
        conditions[i, (i - 1) * p$n + 1:p$n] <- p$weight
    }
    for(i in (m + 1):(p$n + m)) {
        conditions[i, seq(i-m, p$n*m, by = p$n)] <- 1
    }

    Solve(rep(p$value, m), conditions, 
          c(rep("<=", m + p$n)),
          c(rep(p$capacity, m), rep(1, p$n)),
                 types = types, max = max, ...)
}

#### Heurísticas:
TODO

In [48]:
KnapsackMultipleRounding <- function(p, m, InitialSolution = KnapsackMultipleExact, 
                                     RoundingFunction = floor, ...) {
    sol.exact <- InitialSolution(p, m, types = "C", ...)
    
    sol <- list()
    sol$solution = matrix(RoundingFunction(sol.exact$solution), nrow = m, byrow = TRUE)    
    sol$objval   = sum(sol$solution * p$value)
    sol$slack    = p$capacity - colSums(t(sol$solution) * p$weight)
    return(sol)
}

In [49]:
KnapsackMultipleGreedy <- function(p, m, ...) {
    sol <- list(solution = matrix(rep(0, p$n * m), nrow = m), 
                objval = 0, slack = rep(p$capacity, m))

    for(i in 1:m) {
        p.i <- SimpleKnapsackSubSet(p, colSums(sol$solution) == 0)
        if(!is.null(p.i)) {
            s.i <- KnapsackSimpleGreedy(p.i, ...)
            sol$solution[i, colSums(sol$solution) == 0] <- s.i$solution
            sol$objval <- sol$objval + s.i$objval
            sol$slack[i] <- s.i$slack
        }
        
    }
    
    return(sol)
}

In [50]:
KnapsackMultipleLocalSearch <- function(p, InitialSolution = KnapsackMultipleGreedy, ...) {
    
}

### Ejemplos

In [51]:
m <- 3

In [52]:
GenerateMultipleKnapsackSolutions <- function(p, m, ...) {
    sol <- list()
    
    # s <- KnapsackMultipleExact(p, m, ...)
    # sol$exact <- list(solution = matrix(s$solution, ncol = p$n, byrow = TRUE),
    #                   objval   = s$optimum,
    #                   slack    = p$capacity - s$auxiliary$primal[1])

    s <- KnapsackMultipleExact(p, m, types = "C", ...)
    sol$relaxed <- list(solution = matrix(s$solution, ncol = p$n, byrow = TRUE),
                        objval   = s$optimum,
                        slack    = matrix(rep(p$capacity, m) - s$auxiliary$primal[m], 
                                          ncol = m, byrow =TRUE))

    s <- KnapsackMultipleRounding(p, m, ...)
    sol$rounded <- list(solution = s$solution,
                        objval   = s$objval,
                        slack    = matrix(s$slack, ncol = m, byrow = TRUE))

    s <- KnapsackMultipleGreedy(p, m, ...)
    sol$greedy <- list(solution = s$solution,
                       objval   = s$objval,
                        slack    = matrix(s$slack, ncol = m, byrow = TRUE))

    # s <- KnapsackMultipleLocalSearch(p, m, ...)
    # sol$local <- list(solution = matrix(s$solution, ncol = p$n, byrow = TRUE),
    #                   objval   = s$objval,
    #                   slack    = matrix(s$slack, ncol = m, byrow = TRUE))
    
    return(sol)
}

In [53]:
gen10.1 <- ReadSimpleKnapsack(paste0(dataset.folder.simple, "gen10_1.txt"))

In [54]:
GenerateMultipleKnapsackSolutions(gen10.1, m)

$relaxed
$relaxed$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    1    1    1    1    1    0    1    0    0     0     1     1     0     0
[2,]    0    0    0    0    0    0    0    0    1     1     0     0     1     1
[3,]    0    0    0    0    0    1    0    1    0     0     0     0     0     0
     [,15] [,16]     [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25]
[1,]     1     0 0.0000000     0     1     0     0     0     0     0   0.4
[2,]     0     1 0.7692308     0     0     0     1     1     1     1   0.6
[3,]     0     0 0.2307692     1     0     1     0     0     0     0   0.0
     [,26] [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37]
[1,]     1     0     1     0     0     0     1     0     1     1     1     1
[2,]     0     1     0     1     1     1     0     1     0     0     0     0
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,38] [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49]
[1,]     0     0     0     0     0     0     0     1     1     0     1     0
[2,]     0     0     1     1     0     1     1     0     0     0     0     0
[3,]     1     1     0     0     1     0     0     0     0     0     0     1
     [,50]
[1,]     0
[2,]     1
[3,]     0

$relaxed$objval
[1] 2286

$relaxed$slack
     [,1] [,2] [,3]
[1,]  171  171  171


$rounded
$rounded$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    1    1    1    1    1    0    1    0    0     0     1     1     0     0
[2,]    0    0    0    0    0    0    0    0    1     1     0     0     1     1
[3,]    0    0    0    0    0    1    0    1    0     0     0     0     0     0
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     1     0     0     0     1     0     0     0     0     0     0     1
[2,]     0     1     0     0     0     0     1     1     1     1     0     0
[3,]     0     0     0     1     0     1     0     0     0     0     0     0
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     0     1     0     0     0     1     0     1     1     1     1     0
[2,]     1     0     1     1     1     0     1     0     0     0     0     0
[3,]     0     0     0     0     0     0     0     0     0     0     0     1
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]     0     0     0     0     0     0     1     1     0     1     0     0
[2,]     0     1     1     0     1     1     0     0     0     0     0     1
[3,]     1     0     0     1     0     0     0     0     0     0     1     0

$rounded$objval
[1] 2044

$rounded$slack
     [,1] [,2] [,3]
[1,]    6   19  174


$greedy
$greedy$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    1    1    1    1    1    0    0    0    0     1     1     1     0     0
[2,]    0    0    0    0    0    1    1    0    1     0     0     0     0     1
[3,]    0    0    0    0    0    0    0    1    0     0     0     0     1     0
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     1     0     0     0     1     0     0     0     0     0     0     1
[2,]     0     1     0     0     0     0     1     1     1     1     1     0
[3,]     0     0     1     1     0     1     0     0     0     0     0     0
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     0     1     1     0     0     1     1     1     0     1     1     0
[2,]     1     0     0     1     1     0     0     0     1     0     0     0
[3,]     0     0     0     0     0     0     0     0     0     0     0     1
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]     0     0     0     0     0     0     1     1     0     1     0     0
[2,]     0     1     1     0     1     1     0     0     0     0     0     1
[3,]     1     0     0     1     0     0     0     0     1    

##### gen100.1

In [55]:
gen100.1 <- ReadSimpleKnapsack(paste0(dataset.folder.simple, "gen100_1.txt"))

In [56]:
GenerateMultipleKnapsackSolutions(gen100.1, m)

$relaxed
$relaxed$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    1    0    0    1    0    0    0    0    0     0     0     1     0     0
[2,]    0    1    1    0    1    0    0    0    0     0     1     0     0     0
[3,]    0    0    0    0    0    0    1    0    0     1     0     0     0     1
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     0     0     0     0     0     0     0     0     0     0     0     0
[2,]     1     0     0     0     1     0     0     0     0     0     0     0
[3,]     0     0     0     0     0     0     0     0     1     0     1     1
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     0     0     0     0     0     1     0     0     0     0   0.0     0
[2,]     0     1     0     0     0     0     0     1     1     0   0.5     0
[3,]     1     0     1     0     1     0     1     0     0     1   0.5     0
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]     0     0     0     0     0     0     1     1     0     0     0     0
[2,]     0     0     0     0     0     0     0     0     0     1     0     0
[3,]     0     1     0     0     1     1     0     0     0     0     0     1
     [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,58] [,59] [,60] [,61] [,62]
[1,]     0     1     0     0     0     1     0     0     0     0     0     0
[2,]     0     0     0     0     0     0     0     1     0     1     0     0
[3,]     0     0     0     0     0     0     0     0     1     0     0     1
     [,63] [,64] [,65] [,66] [,67] [,68] [,69] [,70] [,71] [,72] [,73] [,74]
[1,]     0     0     0     1     0     0     0     1     0     0     0     0
[2,]     1     0     0     0     0     1     0     0     0     0     0     1
[3,]     0     0     0     0     1     0     0     0     0     0     0     0
     [,75] [,76]     [,77] [,78] [,79] [,80] [,81] [,82] [,83] [,84] [,85]
[1,]     0     1 0.0000000     1     0     0     1     0     0     0     0
[2,]     0     0 0.0000000     0     0     0     0     0     0     0     1
[3,]     0     0 0.4615385     0     1     0     0     1     0     0     0
     [,86] [,87] [,88] [,89] [,90] [,91] [,92] [,93] [,94] [,95] [,96] [,97]
[1,]     0     1     0     0     0     1     1     0     0     0     1     0
[2,]     0     0     1     1     0     0     0     0     0     1     0     0
[3,]     1     0     0     0     0     0     0     0     0     0     0     0
     [,98] [,99] [,100]
[1,]     0     1 0.9375
[2,]     0     0 0.0625
[3,]     0     0 0.0000

$relaxed$objval
[1] 3866.692

$relaxed$slack
     [,1] [,2] [,3]
[1,]    0    0    0


$rounded
$rounded$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    1    0    0    1    0    0    0    0    0     0     0     1     0     0
[2,]    0    1    1    0    1    0    0    0    0     0     1     0     0     0
[3,]    0    0    0    0    0    0    1    0    0     1     0     0     0     1
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     0     0     0     0     0     0     0     0     0     0     0     0
[2,]     1     0     0     0     1     0     0     0     0     0     0     0
[3,]     0     0     0     0     0     0     0     0     1     0     1     1
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     0     0     0     0     0     1     0     0     0     0     0     0
[2,]     0     1     0     0     0     0     0     1     1     0     0     0
[3,]     1     0     1     0     1     0     1     0     0     1     0     0
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]     0     0     0     0     0     0     1     1     0     0     0     0
[2,]     0     0     0     0     0     0     0     0     0     1     0     0
[3,]     0     1     0     0     1     1     0     0     0     0     0     1
     [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,5

TODO(@garciparedes): Analyze results

##### gen100.2

In [57]:
gen100.2 <- ReadSimpleKnapsack(paste0(dataset.folder.simple, "gen100_2.txt"))

In [58]:
GenerateMultipleKnapsackSolutions(gen100.2, m)

$relaxed
$relaxed$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    0    0    0    0    0    1    0    0    0     0     0     0     0     0
[2,]    0    1    0    0    0    0    1    0    0     0     0     1     0     0
[3,]    1    0    0    0    0    0    0    1    0     1     1     0     1     0
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     0     0     1     0     1     0     0     0     0     0     0     0
[2,]     0     0     0     0     0     0     0     0     0     0     0     0
[3,]     0     0     0     0     0     0     1     0     0     0     1     1
     [,27]      [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37]
[1,]     0 0.92857143     0     0     0     0     0     0     0     0     0
[2,]     0 0.07142857     1     0     1     0     0     1     0     0     0
[3,]     1 0.00000000     0     0     0     0     0     0     1     1     1
     [,38] [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47]  [,48] [,49]
[1,]     0     0     0     1     1     0     0     0     0     0 0.0000     1
[2,]     0     0     1     0     0     1     0     1     0     1 0.4375     0
[3,]     1     0     0     0     0     0     0     0     0     0 0.5625     0
     [,50] [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,58] [,59] [,60] [,61]
[1,]     0     0     0     0     0     0     0     0     1     0     1     0
[2,]     0     0     0     0     0     1     0     0     0     1     0     1
[3,]     1     0     0     0     0     0     1     1     0     0     0     0
     [,62] [,63] [,64] [,65] [,66] [,67] [,68] [,69] [,70] [,71] [,72] [,73]
[1,]     1     0     1     0     0     0     1     0     0     1     0     1
[2,]     0     0     0     0     0     0     0     0     0     0     0     0
[3,]     0     0     0     0     0     0     0     0     0     0     1     0
     [,74] [,75] [,76] [,77]     [,78] [,79] [,80] [,81] [,82] [,83] [,84]
[1,]     1     0     0     0 0.0000000     0     0     0     1     0     0
[2,]     0     1     0     0 0.0000000     0     0     0     0     1     0
[3,]     0     0     0     0 0.4166667     0     0     0     0     0     0
     [,85] [,86] [,87] [,88] [,89] [,90] [,91] [,92] [,93] [,94] [,95] [,96]
[1,]     0     0     0     1     0     0     1     0     0     1     0     0
[2,]     0     0     1     0     0     0     0     1     0     0     1     0
[3,]     1     0     0     0     0     1     0     0     0     0     0     1
     [,97] [,98] [,99] [,100]
[1,]     0     0     0      0
[2,]     1     0     0      0
[3,]     0     0     0      1

$relaxed$objval
[1] 3660.833

$relaxed$slack
     [,1] [,2] [,3]
[1,]    0    0    0


$rounded
$rounded$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    0    0    0    0    0    1    0    0    0     0     0     0     0     0
[2,]    0    1    0    0    0    0    1    0    0     0     0     1     0     0
[3,]    1    0    0    0    0    0    0    1    0     1     1     0     1     0
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     0     0     1     0     1     0     0     0     0     0     0     0
[2,]     0     0     0     0     0     0     0     0     0     0     0     0
[3,]     0     0     0     0     0     0     1     0     0     0     1     1
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     0     0     0     0     0     0     0     0     0     0     0     0
[2,]     0     0     1     0     1     0     0     1     0     0     0     0
[3,]     1     0     0     0     0     0     0     0     1     1     1     1
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]     0     0     1     1     0     0     0     0     0     0     1     0
[2,]     0     1     0     0     1     0     1     0     1     0     0     0
[3,]     0     0     0     0     0     0     0     0     0     0     0     1
     [,51] [,52] [,53] [,5

TODO(@garciparedes): Analyze results

##### gen100.3

In [59]:
gen100.3 <- ReadSimpleKnapsack(paste0(dataset.folder.simple, "gen100_3.txt"))

In [60]:
GenerateMultipleKnapsackSolutions(gen100.3, m)

$relaxed
$relaxed$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    0    0    0    0    0    0    1    1    0     0     0     1     0     1
[2,]    1    0    0    0    0    1    0    0    0     1     0     0     1     0
[3,]    0    0    1    0    0    0    0    0    0     0     0     0     0     0
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     1     1     0     1     0     0     0     0     1     0     0     0
[2,]     0     0     0     0     0     0     0     0     0     0     0     0
[3,]     0     0     1     0     0     0     1     0     0     0     1     1
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     0     0     0     0     1     0     0     1     0     1     0     0
[2,]     0     0     1     1     0     0     0     0     0     0     0     0
[3,]     0     1     0     0     0     1     1     0     1     0     1     0
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]     1     0     0     0     0     0     0     0     0     1     0     0
[2,]     0     1     1     1     1     1     0     0     0     0     0     1
[3,]     0     0     0     0     0     0     0     1     0     0     0     0
     [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,58] [,59] [,60] [,61] [,62]
[1,]     0     0     1     0     0     0     0     0     0     0     0     0
[2,]     0     0     0     0     0     0     0     0     0     0     0     0
[3,]     0     1     0     0     0     0     1     0     0     0     0     0
     [,63] [,64] [,65] [,66] [,67] [,68] [,69] [,70]      [,71] [,72] [,73]
[1,]     0     0     0     0     0     0     0     0 0.05555556     0     0
[2,]     0     0     0     1     0     0     1     1 0.94444444     1     1
[3,]     1     0     0     0     1     0     0     0 0.00000000     0     0
     [,74] [,75] [,76] [,77]     [,78] [,79] [,80] [,81] [,82] [,83] [,84]
[1,]     0     1     1     1 0.0000000     0     1     0     0     0     0
[2,]     0     0     0     0 0.1666667     0     0     0     0     0     0
[3,]     0     0     0     0 0.8333333     0     0     0     1     0     0
     [,85] [,86] [,87] [,88] [,89] [,90] [,91]     [,92] [,93] [,94] [,95]
[1,]     0     0     0     0     0     0     0 0.0000000     1     0     0
[2,]     0     0     0     0     0     0     0 0.0000000     0     0     1
[3,]     1     1     0     1     1     0     0 0.6153846     0     0     0
     [,96] [,97] [,98] [,99] [,100]
[1,]     0     1     0     0      1
[2,]     1     0     0     0      0
[3,]     0     0     0     0      0

$relaxed$objval
[1] 3915.846

$relaxed$slack
     [,1] [,2] [,3]
[1,]    0    0    0


$rounded
$rounded$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    0    0    0    0    0    0    1    1    0     0     0     1     0     1
[2,]    1    0    0    0    0    1    0    0    0     1     0     0     1     0
[3,]    0    0    1    0    0    0    0    0    0     0     0     0     0     0
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     1     1     0     1     0     0     0     0     1     0     0     0
[2,]     0     0     0     0     0     0     0     0     0     0     0     0
[3,]     0     0     1     0     0     0     1     0     0     0     1     1
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     0     0     0     0     1     0     0     1     0     1     0     0
[2,]     0     0     1     1     0     0     0     0     0     0     0     0
[3,]     0     1     0     0     0     1     1     0     1     0     1     0
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]     1     0     0     0     0     0     0     0     0     1     0     0
[2,]     0     1     1     1     1     1     0     0     0     0     0     1
[3,]     0     0     0     0     0     0     0     1     0     0     0     0
     [,51] [,5

TODO(@garciparedes): Analyze results

##### gen100.4

In [61]:
gen100.4 <- ReadSimpleKnapsack(paste0(dataset.folder.simple, "gen100_4.txt"))

In [62]:
GenerateMultipleKnapsackSolutions(gen100.4, m)

$relaxed
$relaxed$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    0    1    0    1    0    0    0    0    0     0     0     0     0     1
[2,]    1    0    0    0    1    0    0    0    0     0     0     0     0     0
[3,]    0    0    1    0    0    1    0    0    1     0     1     0     1     0
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     0     0     0     0     0   0.0     0     0     0     0     1     0
[2,]     1     0     0     0     0   0.0     1     0     0     0     0     0
[3,]     0     1     1     1     1   0.5     0     0     0     0     0     0
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     0     0     0     0     0     0     0     0     0     1     0     0
[2,]     0     0     0     1     1     0     1     0     0     0     0     0
[3,]     1     0     0     0     0     0     0     1     0     0     1     1
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]     0     0     0     0     0     0     0     1     0     0     0     0
[2,]     1     0     0     1     0     1     0     0     0     0     0     0
[3,]     0     1     0     0     0     0     1     0     0     0     0     0
     [,51] [,52] [,53] [,54] [,55] [,56] [,57]     [,58] [,59] [,60] [,61]
[1,]     0     1     0     1     0     0     0 0.2631579     0     0     1
[2,]     1     0     0     0     0     1     1 0.7368421     0     0     0
[3,]     0     0     0     0     0     0     0 0.0000000     0     0     0
     [,62] [,63] [,64] [,65] [,66] [,67] [,68] [,69] [,70] [,71] [,72] [,73]
[1,]     1     0     0     0     0     0     0     0     1     0     0     1
[2,]     0     1     0     1     0     0     0     0     0     1     0     0
[3,]     0     0     1     0     0     1     0     0     0     0     1     0
     [,74] [,75] [,76]     [,77] [,78] [,79] [,80] [,81] [,82] [,83] [,84]
[1,]     1     1     0 0.0000000     0     1     0     0     0     0     1
[2,]     0     0     0 0.1764706     0     0     0     0     1     1     0
[3,]     0     0     0 0.8235294     0     0     0     0     0     0     0
     [,85] [,86] [,87] [,88] [,89] [,90] [,91] [,92] [,93] [,94] [,95] [,96]
[1,]     0     1     0     0     1     0     0     0     0     0     0     1
[2,]     0     0     0     0     0     0     0     0     1     0     0     0
[3,]     0     0     1     0     0     1     0     0     0     0     0     0
     [,97] [,98] [,99] [,100]
[1,]     0     0     1      0
[2,]     0     0     0      0
[3,]     0     0     0      1

$relaxed$objval
[1] 3879.5

$relaxed$slack
     [,1] [,2] [,3]
[1,]    0    0    0


$rounded
$rounded$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    0    1    0    1    0    0    0    0    0     0     0     0     0     1
[2,]    1    0    0    0    1    0    0    0    0     0     0     0     0     0
[3,]    0    0    1    0    0    1    0    0    1     0     1     0     1     0
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     0     0     0     0     0     0     0     0     0     0     1     0
[2,]     1     0     0     0     0     0     1     0     0     0     0     0
[3,]     0     1     1     1     1     0     0     0     0     0     0     0
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     0     0     0     0     0     0     0     0     0     1     0     0
[2,]     0     0     0     1     1     0     1     0     0     0     0     0
[3,]     1     0     0     0     0     0     0     1     0     0     1     1
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]     0     0     0     0     0     0     0     1     0     0     0     0
[2,]     1     0     0     1     0     1     0     0     0     0     0     0
[3,]     0     1     0     0     0     0     1     0     0     0     0     0
     [,51] [,52] [,53] [,54] [,55] [

TODO(@garciparedes): Analyze results

##### gen100.5

In [63]:
gen100.5 <- ReadSimpleKnapsack(paste0(dataset.folder.simple, "gen100_5.txt"))

In [64]:
GenerateMultipleKnapsackSolutions(gen100.5, m)

$relaxed
$relaxed$solution
     [,1] [,2] [,3] [,4] [,5] [,6]       [,7] [,8]      [,9] [,10] [,11] [,12]
[1,]    0    0    0    0    0    0 0.00000000    0 0.0000000     0     0     0
[2,]    0    0    0    0    0    0 0.00000000    0 0.2105263     0     0     0
[3,]    0    0    1    0    0    1 0.07142857    0 0.7894737     0     1     0
     [,13] [,14] [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24]
[1,]     0     0     1     0     0     1     0     1     0     0     0     0
[2,]     0     1     0     1     0     0     0     0     0     0     0     0
[3,]     0     0     0     0     0     0     1     0     1     1     0     0
     [,25] [,26] [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36]
[1,]     0   0.5     0     0     0     0     0     0     0     0     0     0
[2,]     0   0.5     0     1     1     1     0     1     0     0     0     0
[3,]     0   0.0     1     0     0     0     0     0     0     0     1     1
     [,37] [,38] [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48]
[1,]     1     0     1     1     0     1     0     1     0     0     0     0
[2,]     0     0     0     0     0     0     0     0     0     0     0     0
[3,]     0     1     0     0     1     0     1     0     0     0     0     0
     [,49] [,50] [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,58] [,59] [,60]
[1,]     0     0     0     0     0     0     0     0     0     0     0     0
[2,]     0     0     0     0     0     0     0     0     0     1     1     0
[3,]     0     0     1     0     1     1     0     0     0     0     0     0
     [,61] [,62] [,63] [,64] [,65] [,66] [,67] [,68] [,69] [,70] [,71] [,72]
[1,]     0     0     0     0     1     1     1     0     0     0     1     0
[2,]     1     0     0     0     0     0     0     1     0     0     0     0
[3,]     0     0     0     0     0     0     0     0     1     0     0     0
     [,73] [,74] [,75] [,76] [,77] [,78] [,79] [,80] [,81] [,82] [,83] [,84]
[1,]     0     1     1     0     0     0     0     0     0     0     0     0
[2,]     1     0     0     0     0     0     1     1     0     1     1     1
[3,]     0     0     0     1     0     1     0     0     0     0     0     0
     [,85] [,86] [,87] [,88] [,89] [,90] [,91] [,92] [,93] [,94] [,95] [,96]
[1,]     1     1     0     1     1     1     0     0     0     0     0     0
[2,]     0     0     0     0     0     0     1     0     0     1     0     0
[3,]     0     0     0     0     0     0     0     0     1     0     0     1
     [,97] [,98] [,99] [,100]
[1,]     0     0     0      0
[2,]     0     0     0      1
[3,]     0     0     1      0

$relaxed$objval
[1] 4042.357

$relaxed$slack
     [,1] [,2] [,3]
[1,]    0    0    0


$rounded
$rounded$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    0    0    0    0    0    0    0    0    0     0     0     0     0     0
[2,]    0    0    0    0    0    0    0    0    0     0     0     0     0     1
[3,]    0    0    1    0    0    1    0    0    0     0     1     0     0     0
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     1     0     0     1     0     1     0     0     0     0     0     0
[2,]     0     1     0     0     0     0     0     0     0     0     0     0
[3,]     0     0     0     0     1     0     1     1     0     0     0     0
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     0     0     0     0     0     0     0     0     0     0     1     0
[2,]     0     1     1     1     0     1     0     0     0     0     0     0
[3,]     1     0     0     0     0     0     0     0     1     1     0     1
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]     1     1     0     1     0     1     0     0     0     0     0     0
[2,]     0     0     0     0     0     0     0     0     0     0     0     0
[3,]     0     0     1     0     1     0     0     0     0     0     0     0
     [,51] [,52] [,53]

TODO(@garciparedes): Analyze results

##### gen200.1

In [65]:
gen200.1 <- ReadSimpleKnapsack(paste0(dataset.folder.simple, "gen200_1.txt"))

In [66]:
GenerateMultipleKnapsackSolutions(gen200.1, m)

$relaxed
$relaxed$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    0    0    1    0    0    0    1    1    0     1     0     0     0     1
[2,]    0    0    0    0    0    0    0    0    0     0     0     1     1     0
[3,]    1    0    0    1    0    1    0    0    0     0     1     0     0     0
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     1     0     0     0     0     1     0     1     0     0     0     1
[2,]     0     0     0     0     0     0     0     0     0     0     1     0
[3,]     0     1     1     0     1     0     1     0     1     1     0     0
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     0     0     0     0     1     0     0     1     0     0     1     0
[2,]     0     0     0     0     0     1     1     0     0     0     0     1
[3,]     1     1     1     1     0     0     0     0     0     0     0     0
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]     0     0     0     0     1     0     0     1     0     0     0     1
[2,]     0     1     1     0     0     0     1     0     1     1     0     0
[3,]     1     0     0     1     0     0     0     0     0     0     0     0
     [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,58] [,59] [,60] [,61] [,62]
[1,]     0     0     1     0     0     0     0     0     0     0     0     1
[2,]     0     0     0     1     0     1     1     0     0     0     1     0
[3,]     1     1     0     0     0     0     0     0     1     1     0     0
     [,63] [,64] [,65] [,66] [,67] [,68] [,69] [,70] [,71] [,72] [,73] [,74]
[1,]     1     0     0     0     1     0     0     1     1     0     0     0
[2,]     0     1     0     0     0     0     0     0     0     0     0     0
[3,]     0     0     1     1     0     0     0     0     0     0     1     1
     [,75] [,76] [,77] [,78] [,79] [,80] [,81] [,82] [,83] [,84] [,85] [,86]
[1,]     0     0     1     1     0     0     1     0     0     0     0     0
[2,]     1     1     0     0     0     1     0     1     0     1     0     1
[3,]     0     0     0     0     1     0     0     0     1     0     0     0
     [,87] [,88] [,89] [,90] [,91] [,92] [,93] [,94] [,95] [,96] [,97] [,98]
[1,]     0     1     1     0     1     1     0     0     0     1     0     0
[2,]     0     0     0     0     0     0     1     0     0     0     1     0
[3,]     0     0     0     1     0     0     0     1     1     0     0     1
     [,99] [,100] [,101] [,102] [,103] [,104] [,105] [,106] [,107] [,108]
[1,]     0      0      0      1      1      0      0      0      1      0
[2,]     1      1      0      0      0      0      0      0      0      0
[3,]     0      0      1      0      0      0      0      1      0      1
     [,109] [,110] [,111] [,112] [,113] [,114] [,115] [,116] [,117] [,118]
[1,]      0      0      0      1      1      0      0      0      0      0
[2,]      0      1      1      0      0      0      1      0      0      1
[3,]      1      0      0      0      0      1      0      1      1      0
     [,119] [,120] [,121] [,122] [,123] [,124] [,125] [,126] [,127] [,128]
[1,]      0      1      0      0      0      0      0      0      1      1
[2,]      0      0      0      1      1      0      1      0      0      0
[3,]      1      0      1      0      0      1      0      1      0      0
     [,129] [,130] [,131] [,132] [,133] [,134] [,135]    [,136] [,137] [,138]
[1,]      1      0      0      0      0      0      1 0.0000000      0      0
[2,]      0      0      1      1      1      0      0 0.6111111      0      1
[3,]      0      1      0      0      0      1      0 0.3888889      1      0
     [,139] [,140] [,141] [,142] [,143] [,144] [,145] [,146] [,147] [,148]
[1,]      0      0      1      1      1      0      1      1      0      0
[2,]      1      0      0      0      0      0      0      0      0      0
[3,]      0      1      0      0      0      0      0      0    

##### gen200.2

In [67]:
gen200.2 <- ReadSimpleKnapsack(paste0(dataset.folder.simple, "gen200_2.txt"))

In [68]:
GenerateMultipleKnapsackSolutions(gen200.2, m)

$relaxed
$relaxed$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    0    0    0    0    1    1    1    0    0     0     0     0     0     0
[2,]    0    1    1    0    0    0    0    0    1     1     1     1     0     0
[3,]    1    0    0    1    0    0    0    1    0     0     0     0     1     1
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     0     1     1     0     0     1     0     0     0     0     1     0
[2,]     1     0     0     1     0     0     1     0     1     1     0     0
[3,]     0     0     0     0     1     0     0     1     0     0     0     1
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     0     0     0     0     0     0     0     0     1     1     0     0
[2,]     0     1     1     1     1     1     1     0     0     0     0     0
[3,]     0     0     0     0     0     0     0     1     0     0     1     1
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]     0     0     0     1     0     0     0     0     0     0     0     1
[2,]     1     1     0     0     0     0     0     1     0     1     1     0
[3,]     0     0     1     0     0     1     1     0     0     0     0     0
     [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,58] [,59] [,60] [,61] [,62]
[1,]     0     0     0     0     0     0     0     1     1     0     1     1
[2,]     0     1     0     0     0     0     0     0     0     0     0     0
[3,]     0     0     1     0     1     1     0     0     0     1     0     0
     [,63] [,64] [,65] [,66] [,67] [,68] [,69] [,70] [,71] [,72] [,73] [,74]
[1,]     0     0     0     0     1     0     0     1     0     0     0     1
[2,]     0     0     1     0     0     0     0     0     0     0     0     0
[3,]     1     0     0     1     0     0     1     0     0     1     0     0
     [,75] [,76] [,77] [,78] [,79] [,80] [,81] [,82] [,83] [,84] [,85] [,86]
[1,]     0     0     1     1     0     1     0     1     1     0     1     0
[2,]     0     0     0     0     0     0     0     0     0     0     0     1
[3,]     0     1     0     0     1     0     0     0     0     1     0     0
     [,87] [,88] [,89] [,90] [,91] [,92] [,93] [,94] [,95] [,96] [,97] [,98]
[1,]     1     0     1     0     0     1     1     0     0     0     1     1
[2,]     0     0     0     0     1     0     0     1     0     1     0     0
[3,]     0     0     0     1     0     0     0     0     1     0     0     0
     [,99] [,100] [,101] [,102] [,103] [,104] [,105] [,106] [,107] [,108]
[1,]     0      1      0      0      0      1      0      1      0      0
[2,]     0      0      0      1      0      0      0      0      0      0
[3,]     1      0      1      0      0      0      1      0      1      1
     [,109] [,110] [,111] [,112] [,113] [,114] [,115] [,116] [,117] [,118]
[1,]      0      0      0      0      0      1      0      1      0      0
[2,]      1      0      0      1      1      0      0      0      0      1
[3,]      0      1      1      0      0      0      1      0      1      0
     [,119] [,120] [,121] [,122] [,123] [,124] [,125] [,126] [,127] [,128]
[1,]      0      1      0      0      0      0      1      1      0      0
[2,]      0      0      1      1      1      1      0      0      1      0
[3,]      1      0      0      0      0      0      0      0      0      1
     [,129] [,130] [,131] [,132] [,133] [,134] [,135] [,136] [,137] [,138]
[1,]      0      0      1      0      1      0      0      0      1      0
[2,]      0      0      0      0      0      0      0      1      0      0
[3,]      1      0      0      1      0      0      1      0      0      1
     [,139] [,140] [,141] [,142] [,143] [,144] [,145] [,146] [,147] [,148]
[1,]      0      0      0      0      1      0      0      0      0      0
[2,]      1      0      0      0      0      1      0      1      0      1
[3,]      0      0      1      1      0      0      1      0      1      0
 

TODO(@garciparedes): Analyze results

##### gen200.3

In [69]:
gen200.3 <- ReadSimpleKnapsack(paste0(dataset.folder.simple, "gen200_3.txt"))

In [70]:
GenerateMultipleKnapsackSolutions(gen200.3, m)

$relaxed
$relaxed$solution
     [,1] [,2] [,3]      [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13]
[1,]    1    0    0 0.0000000    1    1    1    1    0     0     0     0     0
[2,]    0    0    0 0.0000000    0    0    0    0    0     1     0     0     1
[3,]    0    1    0 0.2777778    0    0    0    0    1     0     1     1     0
     [,14] [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25]
[1,]     0     0     1     1     0     0     0     0     1     0     0     0
[2,]     0     0     0     0     1     1     0     1     0     1     0     1
[3,]     1     1     0     0     0     0     1     0     0     0     1     0
     [,26] [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37]
[1,]     0     0     0     1     0     1     0     1     1     0     0     1
[2,]     1     1     0     0     1     0     0     0     0     0     1     0
[3,]     0     0     1     0     0     0     1     0     0     1     0     0
     [,38] [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49]
[1,]     1     0     0     1     0     0     0     0     0     0     0     1
[2,]     0     1     0     0     0     1     1     0     0     0     0     0
[3,]     0     0     1     0     1     0     0     1     0     1     1     0
     [,50] [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,58] [,59] [,60] [,61]
[1,]     0     0     0     1     1     0     1     1     0     0     0     0
[2,]     1     1     0     0     0     0     0     0     1     0     0     1
[3,]     0     0     1     0     0     1     0     0     0     1     1     0
     [,62] [,63] [,64] [,65] [,66] [,67] [,68] [,69] [,70] [,71] [,72] [,73]
[1,]     0     0     0     0     0     0     1     1     0     1     0     0
[2,]     0     1     1     1     1     0     0     0     0     0     0     1
[3,]     1     0     0     0     0     1     0     0     1     0     0     0
     [,74] [,75] [,76] [,77] [,78] [,79] [,80] [,81] [,82] [,83] [,84] [,85]
[1,]     1     1     0     0     0     0     0     0     0     1     1     0
[2,]     0     0     1     0     0     1     0     0     0     0     0     1
[3,]     0     0     0     1     0     0     0     0     1     0     0     0
     [,86] [,87] [,88] [,89] [,90] [,91] [,92] [,93] [,94] [,95] [,96] [,97]
[1,]     1     0     1     0     0     1     0     0     0     0     1     0
[2,]     0     0     0     0     1     0     0     0     1     0     0     0
[3,]     0     0     0     1     0     0     1     1     0     1     0     1
     [,98] [,99] [,100] [,101] [,102] [,103] [,104] [,105] [,106] [,107] [,108]
[1,]     0     1      1      0      0      1      0      0      1      1      0
[2,]     0     0      0      0      0      0      0      1      0      0      0
[3,]     0     0      0      0      0      0      1      0      0      0      1
     [,109] [,110] [,111] [,112] [,113] [,114] [,115] [,116] [,117] [,118]
[1,]      0      0      0      0      1      0      0      0      1      0
[2,]      1      1      1      1      0      0      1      0      0      1
[3,]      0      0      0      0      0      1      0      0      0      0
     [,119] [,120] [,121]     [,122] [,123] [,124] [,125] [,126] [,127] [,128]
[1,]      0      0      0 0.00000000      0      0      0      0      0      0
[2,]      0      0      1 0.92307692      0      1      0      1      1      1
[3,]      1      1      0 0.07692308      1      0      1      0      0      0
     [,129] [,130] [,131] [,132] [,133] [,134] [,135] [,136] [,137] [,138]
[1,]      0      0      0      0      0      0      1      1      1      0
[2,]      1      0      1      0      1      0      0      0      0      0
[3,]      0      1      0      1      0      1      0      0      0      1
     [,139] [,140] [,141] [,142] [,143] [,144] [,145] [,146] [,147] [,148]
[1,]      0      0      0      0      0      0      1      1      0      0
[2,]      0      0      1      1      0      0      0      0      0      1
[3,]      1      1      0      0      0 

TODO(@garciparedes): Analyze results

##### gen200.4

In [71]:
gen200.4 <- ReadSimpleKnapsack(paste0(dataset.folder.simple, "gen200_4.txt"))

In [72]:
GenerateMultipleKnapsackSolutions(gen200.4, m)

$relaxed
$relaxed$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    0    0    0    0    0    1    0    0    0     1     0     0     0     0
[2,]    0    0    1    0    1    0    1    0    0     0     1     1     0     1
[3,]    0    1    0    1    0    0    0    0    1     0     0     0     1     0
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     0     0     1     0     0     1     0     1     0     0     0     1
[2,]     1     1     0     1     0     0     0     0     0     0     0     0
[3,]     0     0     0     0     1     0     1     0     0     1     1     0
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     0     0     0     0     0     0     0     0     0     0     1     0
[2,]     1     0     0     1     0     1     0     1     0     0     0     1
[3,]     0     1     0     0     1     0     1     0     1     1     0     0
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]     0     0     0     0     0     0     0     0     0     1     0     1
[2,]     0     1     1     0     1     0     0     1     0     0     1     0
[3,]     1     0     0     0     0     1     1     0     0     0     0     0
     [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,58] [,59] [,60] [,61] [,62]
[1,]     0     0     0     1     0     0     0     1     0     0     0     0
[2,]     0     0     1     0     0     0     1     0     0     0     0     0
[3,]     1     1     0     0     1     1     0     0     0     1     0     1
     [,63] [,64] [,65] [,66] [,67] [,68] [,69] [,70] [,71] [,72] [,73] [,74]
[1,]     1     0     0     1     1     0     0     0     1     0     0     0
[2,]     0     0     1     0     0     1     1     0     0     0     0     0
[3,]     0     1     0     0     0     0     0     1     0     0     0     0
     [,75] [,76] [,77] [,78] [,79] [,80] [,81] [,82] [,83] [,84] [,85] [,86]
[1,]     0     0     0     1     0     1     0     0     1     0     0     0
[2,]     0     0     0     0     1     0     1     0     0     0     1     1
[3,]     1     1     1     0     0     0     0     1     0     0     0     0
     [,87] [,88] [,89] [,90] [,91] [,92] [,93] [,94] [,95] [,96] [,97] [,98]
[1,]     0     1     0     0     0     1     1     1     0     0     0     0
[2,]     0     0     1     1     0     0     0     0     1     0     0     0
[3,]     1     0     0     0     1     0     0     0     0     1     1     1
     [,99] [,100] [,101]    [,102] [,103] [,104] [,105] [,106] [,107] [,108]
[1,]     1      0      1 0.0000000      0      1      0      0      1      1
[2,]     0      1      0 0.0000000      1      0      1      0      0      0
[3,]     0      0      0 0.5555556      0      0      0      0      0      0
     [,109] [,110] [,111] [,112] [,113] [,114] [,115] [,116] [,117] [,118]
[1,]      0      1      1    0.0      0      1      0      0      0      1
[2,]      1      0      0    0.1      1      0      0      1      0      0
[3,]      0      0      0    0.9      0      0      1      0      0      0
     [,119] [,120] [,121] [,122] [,123] [,124] [,125] [,126] [,127] [,128]
[1,]      0      1      0      1      1      0      0      1      1      1
[2,]      0      0      0      0      0      1      1      0      0      0
[3,]      0      0      1      0      0      0      0      0      0      0
     [,129] [,130] [,131] [,132]    [,133] [,134] [,135] [,136] [,137] [,138]
[1,]      0      0      0      0 0.7692308      0      0      0      0      0
[2,]      1      0      0      1 0.2307692      0      0      0      0      0
[3,]      0      1      0      0 0.0000000      0      1      1      1      1
     [,139] [,140] [,141] [,142] [,143] [,144] [,145] [,146] [,147] [,148]
[1,]      0      1      1      1      0      0      0      1      0      0
[2,]      1      0      0      0      0      0      1      0      0      1
[3,]      0      0      0      0      1      1      

TODO(@garciparedes): Analyze results

##### gen200.5

In [73]:
gen200.5 <- ReadSimpleKnapsack(paste0(dataset.folder.simple, "gen200_5.txt"))

In [74]:
GenerateMultipleKnapsackSolutions(gen200.5, m)

$relaxed
$relaxed$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    0    1    0    0    1    0    0    1    0     0     0     1     0     0
[2,]    1    0    0    1    0    0    1    0    1     0     0     0     1     1
[3,]    0    0    1    0    0    1    0    0    0     1     1     0     0     0
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     1     0     0     1     1     1     0     0     1     0     0     0
[2,]     0     1     1     0     0     0     0     0     0     0     0     0
[3,]     0     0     0     0     0     0     0     0     0     0     1     1
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     1     1     1     0     1     0     1     0     0     1     1     0
[2,]     0     0     0     0     0     0     0     1     0     0     0     1
[3,]     0     0     0     1     0     0     0     0     1     0     0     0
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]     0     0     0     0     1     0     0     0     0     0     0     0
[2,]     0     0     0     1     0     1     0     0     1     0     1     0
[3,]     1     1     0     0     0     0     1     1     0     0     0     1
     [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,58] [,59] [,60] [,61] [,62]
[1,]     0     0     1     1     0     0     0     0     0     1     0     0
[2,]     1     0     0     0     1     1     0     1     0     0     1     0
[3,]     0     0     0     0     0     0     1     0     1     0     0     1
     [,63] [,64] [,65] [,66] [,67] [,68] [,69] [,70] [,71] [,72] [,73] [,74]
[1,]     0     0     1     0     0     0     0     0     0     0     0     1
[2,]     0     1     0     0     1     0     1     0     0     0     0     0
[3,]     1     0     0     1     0     0     0     1     1     1     1     0
     [,75] [,76] [,77] [,78] [,79] [,80] [,81] [,82] [,83] [,84] [,85] [,86]
[1,]     0     1     0     0     0     0     0     0     1     0     1     0
[2,]     1     0     0     0     1     1     0     1     0     0     0     1
[3,]     0     0     1     0     0     0     1     0     0     1     0     0
     [,87] [,88] [,89] [,90] [,91] [,92] [,93] [,94] [,95] [,96] [,97] [,98]
[1,]     0     0     0     0     0     0     0     0     0     0     1     0
[2,]     1     1     0     1     1     0     0     1     0     0     0     0
[3,]     0     0     1     0     0     1     1     0     0     1     0     1
     [,99] [,100] [,101] [,102] [,103] [,104] [,105] [,106] [,107] [,108]
[1,]     1      1      0      0      1      0      0      0      0      0
[2,]     0      0      0      0      0      1      1      0      1      0
[3,]     0      0      0      1      0      0      0      0      0      1
     [,109] [,110] [,111] [,112] [,113] [,114] [,115] [,116]    [,117] [,118]
[1,]      1      0      0      1      1      1      0      0 0.2222222      1
[2,]      0      0      0      0      0      0      1      0 0.7777778      0
[3,]      0      0      0      0      0      0      0      0 0.0000000      0
     [,119] [,120] [,121] [,122] [,123] [,124] [,125] [,126] [,127] [,128]
[1,]      1      1      0      0      1      1      0      1      0      0
[2,]      0      0      0      0      0      0      0      0      0      0
[3,]      0      0      1      1      0      0      1      0      1      1
     [,129] [,130] [,131] [,132] [,133] [,134] [,135] [,136] [,137] [,138]
[1,]      1  0.000      0      0      1      1      1      1      0      1
[2,]      0  0.000      0      0      0      0      0      0      1      0
[3,]      0  0.125      1      1      0      0      0      0      0      0
     [,139] [,140] [,141] [,142] [,143] [,144] [,145] [,146] [,147] [,148]
[1,]      1      0      0      0      1      0      0      0      0      1
[2,]      0      1      0      0      0      0      0      0      1      0
[3,]      0      0      1      1      0      0      1      1    

TODO(@garciparedes): Analyze results

##### gen300.1

In [75]:
gen300.1 <- ReadSimpleKnapsack(paste0(dataset.folder.simple, "gen300_1.txt"))

In [76]:
GenerateMultipleKnapsackSolutions(gen300.1, m)

$relaxed
$relaxed$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    1    1    0    0    1    1    0    1    0     0     0     0     0     1
[2,]    0    0    1    1    0    0    1    0    1     1     1     1     1     0
[3,]    0    0    0    0    0    0    0    0    0     0     0     0     0     0
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     1     1     1     1     1     1     1     0     0     1     1     0
[2,]     0     0     0     0     0     0     0     1     1     0     0     1
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     0     1     0     0     0     0     1     1     0     1     0     0
[2,]     1     0     1     1     1     1     0     0     1     0     1     1
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]     1     0     1     0     0     1     0     1     0     0     0     1
[2,]     0     1     0     1     1     0     1     0     1     1     1     0
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,58] [,59] [,60] [,61] [,62]
[1,]     0     0     0     1     1     1     0     1     1     1     0     0
[2,]     1     1     1     0     0     0     1     0     0     0     1     1
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,63] [,64] [,65] [,66] [,67] [,68] [,69] [,70] [,71] [,72] [,73] [,74]
[1,]     1     0     0     1     0     0     1     1     1     1     1     0
[2,]     0     1     1     0     1     1     0     0     0     0     0     1
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,75] [,76] [,77] [,78] [,79] [,80] [,81] [,82] [,83] [,84] [,85] [,86]
[1,]     0     1     0     1     0     0     1     0     0     1     0     1
[2,]     1     0     1     0     1     1     0     1     1     0     1     0
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,87] [,88] [,89] [,90] [,91] [,92] [,93] [,94] [,95] [,96] [,97] [,98]
[1,]     1     1     0     1     1     0     0     1     1     0     0     1
[2,]     0     0     1     0     0     1     1     0     0     1     1     0
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,99] [,100] [,101] [,102] [,103] [,104] [,105] [,106] [,107] [,108]
[1,]     0      0      0      1      1      1      0      0      0      0
[2,]     1      1      1      0      0      0      1      1      1      1
[3,]     0      0      0      0      0      0      0      0      0      0
     [,109] [,110] [,111] [,112] [,113] [,114] [,115] [,116] [,117] [,118]
[1,]      1      1      1      1      0      1      1      0      0      1
[2,]      0      0      0      0      1      0      0      1      1      0
[3,]      0      0      0      0      0      0      0      0      0      0
     [,119] [,120] [,121] [,122] [,123] [,124] [,125] [,126] [,127] [,128]
[1,]      0      1      0      0      0      1      1      0      1      1
[2,]      1      0      1      1      1      0      0      1      0      0
[3,]      0      0      0      0      0      0      0      0      0      0
     [,129] [,130] [,131] [,132] [,133] [,134] [,135] [,136] [,137] [,138]
[1,]      1      1      1      0      1      0      1      0      1      0
[2,]      0      0      0      1      0      0      0      1      0      1
[3,]      0      0      0      0      0      0      0      0      0      0
     [,139] [,140] [,141] [,142] [,143] [,144] [,145] [,146] [,147] [,148]
[1,]      0      0      0      1      0      0      1      0      0      0
[2,]      1      0      1      0      1      1      0      1      1      1
[3,]      0      0      0      0      0      0      0      0      0      0
 

TODO(@garciparedes): Analyze results

##### gen300.2

In [77]:
gen300.2 <- ReadSimpleKnapsack(paste0(dataset.folder.simple, "gen300_2.txt"))

In [78]:
GenerateMultipleKnapsackSolutions(gen300.2, m)

$relaxed
$relaxed$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    0    1    0    1    0    0    0    1    1     0     0     1     1     1
[2,]    1    0    1    0    1    1    1    0    0     1     1     0     0     0
[3,]    0    0    0    0    0    0    0    0    0     0     0     0     0     0
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     0     0     0     1     1     0     0     0     0     0     0     1
[2,]     1     1     1     0     0     1     1     1     1     1     1     0
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     1     0     1     1     1     0     1     1     0     0     0     0
[2,]     0     1     0     0     0     1     0     0     1     1     1     1
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]     0     0     0     0     1     1     1     0     1     0     0     0
[2,]     1     1     1     1     0     0     0     1     0     1     1     1
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,58] [,59] [,60] [,61] [,62]
[1,]     1     0     0     1     1     1     1     0     1     1     0     0
[2,]     0     1     1     0     0     0     0     1     0     0     1     1
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,63] [,64] [,65] [,66] [,67] [,68] [,69] [,70] [,71] [,72] [,73] [,74]
[1,]     0     0     1     0     0     1     1     1     0     0     0     1
[2,]     1     1     0     0     1     0     0     0     1     1     1     0
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,75] [,76] [,77] [,78] [,79] [,80] [,81] [,82] [,83] [,84] [,85] [,86]
[1,]     0     1     0     1     0     0     1     0     0     1     1     0
[2,]     1     0     1     0     1     1     0     1     1     0     0     1
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,87] [,88] [,89] [,90] [,91] [,92] [,93] [,94] [,95] [,96] [,97] [,98]
[1,]     0     1     1     1     0     1     1     1     0     1     0     1
[2,]     1     0     0     0     1     0     0     0     1     0     1     0
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,99] [,100] [,101] [,102] [,103] [,104] [,105] [,106] [,107] [,108]
[1,]     1      1      0      0      1      1      1      1      0      0
[2,]     0      0      1      0      0      0      0      0      1      1
[3,]     0      0      0      0      0      0      0      0      0      0
     [,109] [,110] [,111] [,112] [,113] [,114] [,115] [,116] [,117] [,118]
[1,]      0      1      1      0      1      0      0      0      0      1
[2,]      1      0      0      1      0      1      1      1      1      0
[3,]      0      0      0      0      0      0      0      0      0      0
     [,119] [,120] [,121] [,122] [,123] [,124] [,125] [,126] [,127] [,128]
[1,]      0      0      1      0      1      0      1      1      0      0
[2,]      0      1      0      1      0      1      0      0      1      1
[3,]      0      0      0      0      0      0      0      0      0      0
     [,129] [,130] [,131] [,132] [,133] [,134] [,135] [,136] [,137] [,138]
[1,]      1      0      1      0      1      0      1      0      0      1
[2,]      0      1      0      1      0      1      0      1      1      0
[3,]      0      0      0      0      0      0      0      0      0      0
     [,139] [,140] [,141] [,142] [,143] [,144] [,145] [,146] [,147] [,148]
[1,]      0      1      0      0      1      0      1      1      1      0
[2,]      1      0      1      1      0      1      0      0      0      1
[3,]      0      0      0      0      0      0      0      0      0      0
 

TODO(@garciparedes): Analyze results

##### gen300.3

In [79]:
gen300.3 <- ReadSimpleKnapsack(paste0(dataset.folder.simple, "gen300_3.txt"))

In [80]:
GenerateMultipleKnapsackSolutions(gen300.3, m)

$relaxed
$relaxed$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    0    0    1    0    0    0    0    1    1     1     0     1     1     0
[2,]    1    1    0    1    1    1    1    0    0     0     1     0     0     1
[3,]    0    0    0    0    0    0    0    0    0     0     0     0     0     0
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     1     0     0     1     1     1     1     0     1     1     0     0
[2,]     0     1     1     0     0     0     0     1     0     0     1     1
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     1     0     1     0     1     0     0     1     0     0     1     0
[2,]     0     1     0     1     0     1     1     0     1     1     0     1
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]     0     0     0     0     0     0     1     0     1   0.5     0     0
[2,]     1     1     1     1     1     1     0     1     0   0.5     1     1
[3,]     0     0     0     0     0     0     0     0     0   0.0     0     0
     [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,58] [,59] [,60] [,61] [,62]
[1,]     1     0     1     0     1     1     1     0     1     0     0     0
[2,]     0     1     0     1     0     0     0     1     0     1     1     1
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,63] [,64] [,65] [,66] [,67] [,68] [,69] [,70] [,71] [,72] [,73] [,74]
[1,]     0     1     1     0     1     0     0     0     0     0     0     1
[2,]     1     0     0     1     0     1     1     1     1     1     1     0
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,75] [,76] [,77] [,78] [,79] [,80] [,81] [,82] [,83] [,84] [,85] [,86]
[1,]     0     1     0     0     0     0     1     1     0     0     1     0
[2,]     1     0     1     1     1     1     0     0     1     1     0     1
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,87] [,88] [,89] [,90] [,91] [,92] [,93] [,94] [,95] [,96] [,97] [,98]
[1,]     0     0     0     0     0     1     0     1     0     1     1     0
[2,]     1     1     1     1     1     0     1     0     1     0     0     1
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,99] [,100] [,101] [,102] [,103] [,104] [,105] [,106] [,107] [,108]
[1,]     1      1      1      1      0      1      1      0      0      0
[2,]     0      0      0      0      1      0      0      1      0      1
[3,]     0      0      0      0      0      0      0      0      0      0
     [,109] [,110] [,111] [,112] [,113] [,114] [,115] [,116] [,117] [,118]
[1,]      1      1      0      1      0      1      1      0      1      1
[2,]      0      0      1      0      1      0      0      1      0      0
[3,]      0      0      0      0      0      0      0      0      0      0
     [,119] [,120] [,121] [,122] [,123] [,124] [,125] [,126] [,127] [,128]
[1,]      1      1      1      1      1      0      0      0      0      1
[2,]      0      0      0      0      0      1      1      1      1      0
[3,]      0      0      0      0      0      0      0      0      0      0
     [,129] [,130] [,131] [,132] [,133] [,134] [,135] [,136] [,137] [,138]
[1,]      1      0      0      1      0      1      1      0      1      1
[2,]      0      1      1      0      1      0      0      1      0      0
[3,]      0      0      0      0      0      0      0      0      0      0
     [,139] [,140] [,141] [,142] [,143] [,144] [,145] [,146] [,147] [,148]
[1,]      0      0      0      1      0      0      0      1      0      1
[2,]      1      1      1      0      1      1      1      0      1      0
[3,]      0      0      0      0      0      0      0      0      0      0
 

TODO(@garciparedes): Analyze results

##### gen300.4

In [81]:
gen300.4 <- ReadSimpleKnapsack(paste0(dataset.folder.simple, "gen300_4.txt"))

In [82]:
GenerateMultipleKnapsackSolutions(gen300.4, m)

$relaxed
$relaxed$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    0    0    1    1    0    0    0    1    0     1     0     0     1     0
[2,]    1    1    0    0    1    1    1    0    1     0     1     1     0     0
[3,]    0    0    0    0    0    0    0    0    0     0     0     0     0     0
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     1     1     1     1     0     1     0     0     1     1     1     0
[2,]     0     0     0     0     1     0     1     1     0     0     0     1
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,27]      [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37]
[1,]     0 0.05555556     0     1     0     1     0     0     0     0     0
[2,]     1 0.94444444     1     0     1     0     1     1     1     1     1
[3,]     0 0.00000000     0     0     0     0     0     0     0     0     0
     [,38] [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49]
[1,]     1     1     1     0     1     1     0     0     0     0     0     0
[2,]     0     0     0     1     0     0     1     1     1     1     1     1
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,50] [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,58] [,59] [,60] [,61]
[1,]     0     1     1     0     1     0     0     1     1     0     0     0
[2,]     1     0     0     1     0     1     1     0     0     1     1     1
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,62] [,63] [,64] [,65] [,66] [,67] [,68] [,69] [,70] [,71] [,72] [,73]
[1,]     1     0     0     0     1     1     1     1     0     1     0     0
[2,]     0     1     1     1     0     0     0     0     1     0     1     1
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,74] [,75] [,76] [,77] [,78] [,79] [,80] [,81] [,82] [,83] [,84] [,85]
[1,]     1     0     0     0     0     1     0     0     0     0     0     0
[2,]     0     1     1     1     1     0     1     1     1     1     1     1
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,86] [,87] [,88] [,89] [,90] [,91] [,92] [,93] [,94] [,95] [,96] [,97]
[1,]     1     0     1     0     1     1     0     0     0     1     1     0
[2,]     0     1     0     1     0     0     1     1     1     0     0     1
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,98] [,99] [,100] [,101] [,102] [,103] [,104] [,105] [,106] [,107] [,108]
[1,]     1     1      0      0      0      0      0      1      0      0      1
[2,]     0     0      1      1      1      1      1      0      1      1      0
[3,]     0     0      0      0      0      0      0      0      0      0      0
     [,109] [,110] [,111] [,112] [,113] [,114] [,115] [,116] [,117] [,118]
[1,]      1      0      1      0      0      0      1      1      1      1
[2,]      0      1      0      1      1      1      0      0      0      0
[3,]      0      0      0      0      0      0      0      0      0      0
     [,119] [,120] [,121] [,122] [,123] [,124] [,125] [,126] [,127] [,128]
[1,]      1      0      0      0      0      0      1      1      0      0
[2,]      0      0      1      1      1      1      0      0      1      1
[3,]      0      0      0      0      0      0      0      0      0      0
     [,129] [,130] [,131] [,132] [,133] [,134] [,135] [,136] [,137] [,138]
[1,]      1      1      0      0      0      0      1      1      1      1
[2,]      0      0      1      1      1      1      0      0      0      0
[3,]      0      0      0      0      0      0      0      0      0      0
     [,139] [,140] [,141] [,142] [,143] [,144] [,145] [,146] [,147] [,148]
[1,]      0      1      0      0      0      1      1      1      1      0
[2,]      1      0      1      1      1      0      0      0      0      1
[3,]      0      0      0      0      0      0      0   

TODO(@garciparedes): Analyze results

##### gen300.5

In [83]:
gen300.5 <- ReadSimpleKnapsack(paste0(dataset.folder.simple, "gen300_5.txt"))

In [84]:
GenerateMultipleKnapsackSolutions(gen300.5, m)

$relaxed
$relaxed$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    1    0    1    1    1    0    0    1    1     1     1     1     0     1
[2,]    0    1    0    0    0    1    1    0    0     0     0     0     1     0
[3,]    0    0    0    0    0    0    0    0    0     0     0     0     0     0
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     0     1     1     0     1     1     0     0     0     1     0     0
[2,]     1     0     0     1     0     0     1     1     1     0     1     1
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     1     1     0     0     1     1     1     0     1     0     0     0
[2,]     0     0     1     1     0     0     0     1     0     1     1     1
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]     0     0     1     1     1     1     0     1     0     0     0     1
[2,]     1     1     0     0     0     0     1     0     1     1     1     0
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,58] [,59] [,60] [,61] [,62]
[1,]     1     0     0     1     0     1     1     1     1     0     0     0
[2,]     0     1     1     0     1     0     0     0     0     1     0     1
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,63] [,64] [,65] [,66] [,67] [,68] [,69] [,70] [,71] [,72] [,73] [,74]
[1,]     1     1     1     1     0     1     1     1     1     1     0     1
[2,]     0     0     0     0     1     0     0     0     0     0     1     0
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,75] [,76] [,77] [,78] [,79] [,80] [,81] [,82] [,83] [,84] [,85] [,86]
[1,]     0     0     1     0     1     1     1     1     0     1     1     0
[2,]     1     1     0     1     0     0     0     0     1     0     0     1
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,87] [,88] [,89] [,90] [,91] [,92] [,93] [,94] [,95] [,96] [,97] [,98]
[1,]     0     0     0     1     1     1     0     0     0     1     0     0
[2,]     1     1     0     0     0     0     1     1     1     0     1     1
[3,]     0     0     0     0     0     0     0     0     0     0     0     0
     [,99] [,100] [,101] [,102] [,103] [,104] [,105] [,106] [,107] [,108]
[1,]     1      1      0      1      1      1      0      1      1      0
[2,]     0      0      1      0      0      0      1      0      0      1
[3,]     0      0      0      0      0      0      0      0      0      0
     [,109] [,110] [,111] [,112] [,113] [,114] [,115] [,116] [,117] [,118]
[1,]      1      0      1      1      1      1      0      0      0      0
[2,]      0      1      0      0      0      0      1      1      1      1
[3,]      0      0      0      0      0      0      0      0      0      0
     [,119] [,120] [,121] [,122] [,123] [,124] [,125] [,126] [,127] [,128]
[1,]      1      0      0      0      0      0      1      0      0      1
[2,]      0      1      1      1      0      1      0      1      1      0
[3,]      0      0      0      0      0      0      0      0      0      0
     [,129] [,130] [,131] [,132] [,133] [,134] [,135] [,136] [,137] [,138]
[1,]      1      1      0      0      0      0      0      1      1      1
[2,]      0      0      1      1      1      1      1      0      0      0
[3,]      0      0      0      0      0      0      0      0      0      0
     [,139] [,140] [,141] [,142] [,143] [,144] [,145] [,146] [,147] [,148]
[1,]      0      1      0      1      1      0      1      1      0      0
[2,]      1      0      1      0      0      1      0      0      1      1
[3,]      0      0      0      0      0      0      0      0      0      0
 

TODO(@garciparedes): Analyze results

### Mochila Multidimensional

#### Descripción:
TODO

#### Modelo:
TODO

In [85]:
KnapsackMultidimensionalExact <- function(p, types = "B", max = TRUE, ...) {
    Solve(p$value, rbind(p$weight, diag(p$n)), rep("<=", length(p$capacity) + p$n), 
          c(p$capacity, rep(1, p$n)), types = types, max = max, ...)
}

#### Heurísticas:
TODO

In [86]:
KnapsackMultidimensionalRounding <- function(p, m, InitialSolution = KnapsackMultidimensionalExact, 
                                     RoundingFunction = floor, ...) {
    sol.exact <- InitialSolution(p, types = "C", ...)
    
    sol <- list()
    sol$solution = RoundingFunction(sol.exact$solution)
    sol$objval   = sum(sol$solution * p$value)
    sol$slack    = p$capacity - colSums(sol$solution * t(p$weight))
    return(sol)
}

In [87]:
MultidimensionalCriterionRelative <- function(g, p, sol, dim.weight = rep(1, p$m)) {
    if (!is.null(g)) {
        g
    } else {
        p$value / colSums(dim.weight * p$weight)
    }
}

In [88]:
MultidimensionalCriterionShortage <- function(g, p, sol, dim.weight = rep(1, p$m)) {
    MultidimensionalCriterionRelative(g, p, sol, dim.weight = 1  / sol$slack)
}

In [89]:
MultidimensionalCriterionRelaxed <- function(g, p, sol) {
    if (!is.null(g)) {
        g
    } else {
        KnapsackMultidimensionalExact(p, types = "C" )$solution
    }
}

In [90]:
MultidimensionalCriterionSlack <- function(g, p, sol) {
    if (!is.null(g)) {
        g
    } else {
        MultidimensionalCriterionRelative(g, p, sol, 
            dim.weight = KnapsackMultidimensionalExact(p, types = "C" )$auxiliary$primal[1:p$m]
        )
    }
}

In [91]:
KnapsackMultidimensionalGreedy <- function(p, Criterion = MultidimensionalCriterionRelative, ...) {
        sol <- list(solution = rep(0, p$n), 
                objval = 0, slack = p$capacity)
    
    g <- Criterion(NULL, p, sol)
    i <- which(g == max(g[sol$solution == 0]) & sol$solution == 0)[1]
    
    while (!is.na(i) && all(sol$slack - p$weight[,i] > 0)) {
        sol$solution[i] <- 1
        sol$objval <- sol$objval + p$value[i]
        sol$slack <- sol$slack - p$weight[,i]
        g <- Criterion(g, p, sol)
        if (any(sol$solution == 0)) {
            i <- which(g == max(g[sol$solution == 0]) & sol$solution == 0)[1]
        } else {
            i <- NA
        }
    }
    return(sol)
}

In [92]:
KnapsackMultidimensionalLocalSearchBest <- function(p, sol, idx0, idx1) {
    best = list(v = 0, i = 0, j = 0)
    for (i in idx0) {
        for (j in idx1) {
            if (all(sol$slack + p$weight[, j] - p$weight[, i] > 0)) {
                v.temp <- p$value[i] - p$value[j]
                if(best$v < v.temp) {
                    best$v <- v.temp
                    best$i <- i
                    best$j <- j
                }   
            }
        }
    }
    return(best)
}

In [93]:
KnapsackMultidimensionalLocalSearch <- function(p, InitialSolution = KnapsackMultidimensionalGreedy, ...) {
    sol <- InitialSolution(p, ...)
    
    idx0 <- which(sol$solution == 0)
    idx1 <- which(sol$solution == 1)
    
    best <- KnapsackMultidimensionalLocalSearchBest(p, sol, idx0, idx1)
    while (best$v > 0) {
        sol$solution[best$i] <- 1
        sol$solution[best$j] <- 0
        sol$slack <- sol$slack + p$weight[, best$j] - p$weight[, best$i]
        sol$objval <- sol$objval + best$v
        
        idx0 <- c(idx0[idx0 != best$i], best$j)
        idx1 <- c(idx1[idx1 != best$j], best$i)
        
        best <- KnapsackMultidimensionalLocalSearchBest(p, sol, idx0, idx1)
    }
    return(sol)    
}

### Ejemplos

In [94]:
GenerateMultidimensionalKnapsackSolutions <- function(p, ...) {
    sol <- list()
    
    s <- KnapsackMultidimensionalExact(p, ...)
    sol$exact <- list(solution = matrix(s$solution, ncol = p$n, byrow = TRUE),
                      objval   = s$optimum,
                      slack    = matrix(p$capacity - colSums(s$solution * t(p$weight)), 
                                        ncol = p$m, byrow = TRUE))

    s <- KnapsackMultidimensionalExact(p, types = "C", ...)
    sol$relaxed <- list(solution = matrix(s$solution, ncol = p$n, byrow = TRUE),
                        objval   = s$optimum,
                        slack    = matrix(p$capacity - colSums(s$solution * t(p$weight)), 
                                          ncol = p$m, byrow = TRUE))

    s <- KnapsackMultidimensionalRounding(p, ...)
    sol$rounded <- list(solution = matrix(s$solution, ncol = p$n, byrow = TRUE),
                        objval   = s$objval,
                        slack    = matrix(s$slack, ncol = p$m, byrow = TRUE))

    s <- KnapsackMultidimensionalGreedy(p, ...)
    sol$greedy <- list(solution = matrix(s$solution, ncol = p$n, byrow = TRUE),
                       objval   = s$objval,
                       slack    = matrix(s$slack, ncol = p$m, byrow = TRUE))

    s <- KnapsackMultidimensionalLocalSearch(p, ...)
    sol$local <- list(solution = matrix(s$solution, ncol = p$n, byrow = TRUE),
                      objval   = s$objval, 
                      slack    = matrix(s$slack, ncol = p$m, byrow = TRUE))
    
    return(sol)
}

##### mknap1.2

In [95]:
mknap1.2 <- ReadMultidimensionalKnapsack(paste0(dataset.folder.multi, "mknap1_2.dat"))

In [96]:
GenerateMultidimensionalKnapsackSolutions(mknap1.2)

0,1,0,1,1,0,0,1,0,1
53,1,41,58,59,50,36,60,40,10
0.7745358,1,0.43687,0.8668435,0,1,0,1,0,0
58.45358,0,2.842171e-14,25.57825,33.23607,32.23077,33.70557,10.16446,25.08223,0
0,1,0,0,0,1,0,1,0,0
291,315,155,276,327,352,133,236,276,276
0,1,1,0,0,1,0,1,0,0
191,185,105,206,257,282,113,156,196,196
0,1,0,1,0,0,0,1,0,1
55,3,45,62,63,54,38,66,46,16


##### mknap1.3

In [97]:
mknap1.3 <- ReadMultidimensionalKnapsack(paste0(dataset.folder.multi, "mknap1_3.dat"))

In [98]:
GenerateMultidimensionalKnapsackSolutions(mknap1.3)

$exact
$exact$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    1    1    0    1    0    1    1    0    1     1     0     0     0     1
     [,15]
[1,]     1

$exact$objval
[1] 4015

$exact$slack
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10]
[1,]   35   35   12   33   53   71    4    4   19    13


$relaxed
$relaxed$solution
     [,1] [,2] [,3] [,4]      [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13]
[1,]    1    1    1    1 0.3041237    1    1    0    1     1     0     0     0
         [,14] [,15]
[1,] 0.9474227     1

$relaxed$objval
[1] 4127.887

$relaxed$slack
         [,1] [,2]     [,3]     [,4]     [,5]     [,6]     [,7] [,8]     [,9]
[1,] 7.020619    0 2.969072 19.45361 35.41237 53.67526 5.463918    0 6.963918
        [,10]
[1,] 1.489691


$rounded
$rounded$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    1    1    1    1    0    1    1    0    0     1     0     0     0     0
     [,15]
[1,]     1

$rounded$objval
[1] 2645

$rounded$slack
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10]
[1,]  170  210   40   75   94  117   73  109  130   134


$greedy
$greedy$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    1    1    1    0    0    1    1    1    1     1     0     0     0     1
     [,15]
[1,]     1

$greedy$objval
[1] 3825

$greedy$slack
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10]
[1,]   87   97   25   62   89  110    7    7   28    24


$local
$local$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    1    1    1    0    1    1    1    0    1     1     0     0     0     1
     [,15]
[1,]     1

$local$objval
[1] 4005

$local$slack
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10]
[1,]   32   22    8   37   56   79    9    5   11    10

##### mknap1.4

In [99]:
mknap1.4 <- ReadMultidimensionalKnapsack(paste0(dataset.folder.multi, "mknap1_4.dat"))

In [100]:
GenerateMultidimensionalKnapsackSolutions(mknap1.4)

$exact
$exact$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    1    0    0    0    0    0    0    0    0     1     0     0     0     1
     [,15] [,16] [,17] [,18] [,19] [,20]
[1,]     1     1     1     1     1     1

$exact$objval
[1] 6120

$exact$slack
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10]
[1,]   53    1   60   47   45   54   24   28   15     0


$relaxed
$relaxed$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9]     [,10] [,11] [,12] [,13]
[1,]    1    1    0    0    0    0    0    0    1 0.4333333     0     0     0
     [,14] [,15] [,16] [,17] [,18] [,19] [,20]
[1,]     1     1  0.95     1     1     1     1

$relaxed$objval
[1] 6155.333

$relaxed$slack
     [,1] [,2]     [,3] [,4]     [,5]     [,6] [,7]  [,8]  [,9] [,10]
[1,] 53.5    0 49.93333 34.1 33.18333 42.43333 27.4 24.45 16.75     0


$rounded
$rounded$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    1    1    0    0    0    0    0    0    0     0     0     0     0     1
     [,15] [,16] [,17] [,18] [,19] [,20]
[1,]     1     0     1     1     1     1

$rounded$objval
[1] 5440

$rounded$slack
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10]
[1,]  149  137   68   72   78   92   30   47   54    42


$greedy
$greedy$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    1    1    0    0    0    0    0    0    0     0     0     0     0     1
     [,15] [,16] [,17] [,18] [,19] [,20]
[1,]     1     1     1     1     1     1

$greedy$objval
[1] 5760

$greedy$slack
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10]
[1,]  119   77   68   62   63   72   30   42   39    22


$local
$local$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    1    1    0    0    0    0    0    1    0     1     0     0     0     1
     [,15] [,16] [,17] [,18] [,19] [,20]
[1,]     1     0     1     0     1     1

$local$objval
[1] 6060

$local$slack
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10]
[1,]   50    2   54   48   47   59   18   21   19     4

##### mknap1.6

In [101]:
mknap1.6 <- ReadMultidimensionalKnapsack(paste0(dataset.folder.multi, "mknap1_6.dat"))

In [102]:
GenerateMultidimensionalKnapsackSolutions(mknap1.6)

$exact
$exact$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    1    1    0    1    0    1    0    1    1     0     1     0     1     0
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     1     1     1     1     1     1     0     0     1     0     1     0
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     1     1     1     0     1     1     0     1     1     1     1     1
     [,39]
[1,]     1

$exact$objval
[1] 10618

$exact$slack
     [,1] [,2] [,3] [,4] [,5]
[1,]    3    4    7   73    0


$relaxed
$relaxed$solution
     [,1]      [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13]
[1,]    1 0.5450927    0    1    0    1    0    1    1     0     1     0     1
     [,14] [,15] [,16] [,17]     [,18] [,19] [,20]     [,21] [,22] [,23] [,24]
[1,]     1     1     1     1 0.6961943     1     1 0.5618935     0     1     0
     [,25] [,26] [,27] [,28] [,29] [,30] [,31] [,32]     [,33] [,34] [,35]
[1,]     1     1     1     1     1     1     1     1 0.2927197     1     1
     [,36] [,37] [,38] [,39]
[1,]     1     1     1     1

$relaxed$objval
[1] 10672.35

$relaxed$slack
     [,1] [,2]         [,3]     [,4] [,5]
[1,]    0    0 1.136868e-13 79.77673    0


$rounded
$rounded$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    1    0    0    1    0    1    0    0    1     0     1     0     1     0
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     0     1     1     0     1     1     0     0     1     0     1     0
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     1     1     1     1     1     0     0     1     1     1     1     1
     [,39]
[1,]     1

$rounded$objval
[1] 8231

$rounded$slack
     [,1] [,2] [,3] [,4] [,5]
[1,]  133  132  142  204   97


$greedy
$greedy$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    1    0    0    0    0    1    0    0    0     0     0     1     1     1
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     1     1     1     0     1     1     0     0     1     1     0     1
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     1     1     1     1     1     1     1     1     1     1     1     1
     [,39]
[1,]     1

$greedy$objval
[1] 8771

$greedy$slack
     [,1] [,2] [,3] [,4] [,5]
[1,]   77  129  192  254  110


$local
$local$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    1    1    0    1    0    1    1    1    1     0     0     0     0     0
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     1     1     1     1     1     0     0     0     1     1     0     1
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     1     1     1     0     1     1     0     1     1     1     1     1
     [,39]
[1,]     1

$local$objval
[1] 10448

$local$slack
     [,1] [,2] [,3] [,4] [,5]
[1,]    1    7   34   96    2

##### mknap1.7

In [103]:
mknap1.7 <- ReadMultidimensionalKnapsack(paste0(dataset.folder.multi, "mknap1_7.dat"))

In [104]:
GenerateMultidimensionalKnapsackSolutions(mknap1.7)

$exact
$exact$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    0    0    0    1    0    1    0    1    1     0     1     1     1     0
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     1     1     1     0     1     1     0     0     1     0     1     1
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     1     1     1     0     1     1     0     1     1     1     1     1
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]     1     1     1     1     1     1     0     0     1     1     1     1

$exact$objval
[1] 16537

$exact$slack
     [,1] [,2] [,3] [,4] [,5]
[1,]    0   11    1   78    0


$relaxed
$relaxed$solution
     [,1]      [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13]
[1,]    0 0.1324864    0    1    0    1    0    1    1     0     1     0     1
     [,14] [,15] [,16] [,17] [,18] [,19] [,20]     [,21] [,22] [,23] [,24]
[1,]     1     1     1     1     0     1     1 0.3974592     0     1     0
         [,25] [,26] [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35]
[1,] 0.9891107     1     1     1     1     1     1     1     0     1     1
         [,36] [,37] [,38] [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46]
[1,] 0.5598911     1     1     1     1     1     1     1     0     1     0
     [,47] [,48] [,49] [,50]
[1,]     1     1     1     1

$relaxed$objval
[1] 16612.82

$relaxed$slack
     [,1]          [,2] [,3]     [,4]         [,5]
[1,]    0 -7.958079e-13    0 66.89655 1.136868e-13


$rounded
$rounded$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    0    0    0    0    0    0    0    1    1     0     1     0     1     1
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     1     1     1     0     0     0     0     0     1     0     0     0
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     1     0     1     0     1     1     0     0     1     0     1     1
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]     0     1     1     1     1     0     1     0     1     1     1     0

$rounded$objval
[1] 14536

$rounded$slack
     [,1] [,2] [,3] [,4] [,5]
[1,]  114  106  140  196   92


$greedy
$greedy$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    1    0    0    0    0    1    0    0    0     0     0     0     1     1
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     1     1     1     0     1     1     0     0     1     1     0     1
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     1     1     1     1     1     1     1     1     1     1     0     0
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]     1     1     1     1     1     0     1     0     1     1     1     1

$greedy$objval
[1] 15614

$greedy$slack
     [,1] [,2] [,3] [,4] [,5]
[1,]    5   75  150  220   55


$local
$local$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    1    0    0    1    0    1    0    1    0     0     0     0     1     0
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     1     1     1     1     1     1     0     0     1     1     0     1
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     1     1     1     0     1     1     0     1     1     1     1     0
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]     1     1     1     1     1     0     1     0     1     1     0     1

$local$objval
[1] 16371

$local$slack
     [,1] [,2] [,3] [,4] [,5]
[1,]    1   36   10  132    1

##### mknapcb1.1

In [105]:
mknapcb1.1 <- ReadMultidimensionalKnapsack(paste0(dataset.folder.multi, "mknapcb1_1.dat"))

In [106]:
GenerateMultidimensionalKnapsackSolutions(mknapcb1.1)

$exact
$exact$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    0    1    0    1    0    0    1    0    1     0     1     0     0     0
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     0     0     0     0     1     0     0     0     0     1     0     1
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     1     0     1     1     0     1     0     0     0     0     0     0
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]     0     0     0     0     0     1     0     0     0     0     0     1
     [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,58] [,59] [,60] [,61] [,62]
[1,]     0     0     0     0     0     0     1     0     0     0     0     1
     [,63] [,64] [,65] [,66] [,67] [,68] [,69] [,70] [,71] [,72] [,73] [,74]
[1,]     1     0     0     1     0     0     1     0     1     0     0     1
     [,75] [,76] [,77] [,78] [,79] [,80] [,81] [,82] [,83] [,84] [,85] [,86]
[1,]     0     0     1     0     1     0     0     0     0     0     1     1
     [,87] [,88] [,89] [,90] [,91] [,92] [,93] [,94] [,95] [,96] [,97] [,98]
[1,]     0     0     0     0     0     1     1     0     0     1     0     0
     [,99] [,100]
[1,]     1      0

$exact$objval
[1] 24381

$exact$slack
     [,1] [,2] [,3] [,4] [,5]
[1,]  105   13  175  125   48


$relaxed
$relaxed$solution
     [,1] [,2] [,3]     [,4]     [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12]
[1,]    0    1    0 0.613248 0.472813    0    1    0    1     0     1     0
     [,13] [,14] [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24]
[1,]     1     0     0     0     0     1     1     0     0     0     0     1
     [,25]      [,26] [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34]     [,35]
[1,]     0 0.07578752     1     0     1     1     0     1     0     0 0.3398414
     [,36] [,37] [,38] [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47]
[1,]     0     0     0     0     0     0     0     0     1     0     0     0
     [,48] [,49] [,50] [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,58] [,59]
[1,]     0     0     1     0     0     0     0     0     0     1     0     0
     [,60] [,61] [,62] [,63] [,64] [,65] [,66] [,67] [,68] [,69] [,70] [,71]
[1,]     0     0     1     1     0     0     1     0     0     1     0     1
     [,72] [,73] [,74] [,75] [,76] [,77] [,78] [,79] [,80] [,81] [,82] [,83]
[1,]     0     0     1     0     0     1     0     1     0     0     0     0
     [,84] [,85] [,86] [,87] [,88] [,89] [,90] [,91]      [,92] [,93] [,94]
[1,]     0     1     1     0     0     0     0     0 0.08123275     1     0
     [,95] [,96] [,97] [,98] [,99] [,100]
[1,]     0     1     0     0     1      0

$relaxed$objval
[1] 24585.9

$relaxed$slack
              [,1]         [,2] [,3]          [,4]          [,5]
[1,] -1.818989e-12 7.275958e-12    0 -1.818989e-12 -3.637979e-12


$rounded
$rounded$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    0    1    0    0    0    0    0    0    0     0     0     0     1     0
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     0     0     0     1     1     0     0     0     0     0     0     0
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     1     0     1     1     0     1     0     0     0     0     0     0
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]     0     0     0     0     0     0     0     0     0     0     0     1
     [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,58] [,59] [,60] [,61] [,62]
[1,]     0     0     0     0     0     0     1     0     0     0     0     1
     [,63] [,64] [,65] [,66] [,67] [,68] [,69] [,70] [,71] [,72] [,73] [,74]
[1,]     1     0     0     1     0     0     0     0     0     0     0     0
     [,75] [,76] [,77] [,78] [,79] [,80] [,81] [,82] [,83] [,84] [,85] [,86]
[1,]     0     0     1     0     0     0     0

##### mknapcb1.2

In [107]:
mknapcb1.2 <- ReadMultidimensionalKnapsack(paste0(dataset.folder.multi, "mknapcb1_2.dat"))

In [108]:
GenerateMultidimensionalKnapsackSolutions(mknapcb1.2)

$exact
$exact$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    0    0    0    1    0    0    0    0    0     0     1     0     0     0
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     0     0     0     0     1     0     1     0     0     0     0     0
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     0     1     1     0     0     0     0     0     1     0     1     0
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]     0     0     0     1     1     0     0     1     0     0     1     1
     [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,58] [,59] [,60] [,61] [,62]
[1,]     0     0     0     1     0     0     1     1     1     0     0     1
     [,63] [,64] [,65] [,66] [,67] [,68] [,69] [,70] [,71] [,72] [,73] [,74]
[1,]     1     0     1     0     0     0     0     0     0     0     0     1
     [,75] [,76] [,77] [,78] [,79] [,80] [,81] [,82] [,83] [,84] [,85] [,86]
[1,]     1     0     0     0     0     0     0     1     0     0     0     0
     [,87] [,88] [,89] [,90] [,91] [,92] [,93] [,94] [,95] [,96] [,97] [,98]
[1,]     0     0     1     0     1     1     0     1     0     1     0     0
     [,99] [,100]
[1,]     0      1

$exact$objval
[1] 24274

$exact$slack
     [,1] [,2] [,3] [,4] [,5]
[1,]    6  287  151  129  196


$relaxed
$relaxed$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    0    0    0    1    0    0    0    0    0     0     1     0     0     0
     [,15] [,16] [,17] [,18]     [,19] [,20] [,21] [,22] [,23] [,24] [,25]
[1,]     0     0     0     0 0.3887279     0     1     0     0     0     0
     [,26] [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35]    [,36] [,37]
[1,]     0     0     1     1     0     0     0     0     0     1 0.537623     1
         [,38] [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48]
[1,] 0.5584857     0     1     0     1     1     0     0     1     0     0
     [,49] [,50] [,51] [,52] [,53] [,54] [,55] [,56] [,57]     [,58]     [,59]
[1,]     1     1     0     0     0     1     0     0     1 0.5634552 0.1048908
     [,60] [,61] [,62] [,63] [,64] [,65] [,66] [,67] [,68] [,69] [,70] [,71]
[1,]     0     0     1     1     0     1     0     0     0     0     0     0
     [,72] [,73] [,74] [,75] [,76] [,77] [,78] [,79] [,80] [,81] [,82] [,83]
[1,]     0     0     1     1     0     0     0     0     0     0     1     0
     [,84] [,85] [,86] [,87] [,88] [,89] [,90] [,91] [,92] [,93] [,94] [,95]
[1,]     0     0     0     0     0     0     0     1     1     1     1     0
     [,96] [,97] [,98] [,99] [,100]
[1,]     1     0     0     0      1

$relaxed$objval
[1] 24538.21

$relaxed$slack
             [,1]         [,2] [,3] [,4]          [,5]
[1,] 7.275958e-12 1.091394e-11    0    0 -1.818989e-12


$rounded
$rounded$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    0    0    0    0    0    0    0    0    0     0     1     0     0     0
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     0     0     0     0     0     0     1     0     0     0     0     0
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     0     0     1     0     0     0     0     0     1     0     1     0
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]     0     1     0     1     1     0     0     1     0     0     1     1
     [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,58] [,59] [,60] [,61] [,62]
[1,]     0     0     0     1     0     0     0     0     0     0     0     1
     [,63] [,64] [,65] [,66] [,67] [,68] [,69] [,70] [,71] [,72] [,73] [,74]
[1,]     0     0     0     0     0     0     0     0     0     0     0     1
     [,75] [,76] [,77] [,78] [,79] [,80] [,81] [,82] [,83] [,84] [,85] [,86]
[1,]     1     0     0     0     0     0     0     0     0     0     0 

##### mknapcb1.3

In [109]:
mknapcb1.3 <- ReadMultidimensionalKnapsack(paste0(dataset.folder.multi, "mknapcb1_3.dat"))

In [110]:
GenerateMultidimensionalKnapsackSolutions(mknapcb1.3)

$exact
$exact$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    0    0    0    0    1    0    0    0    0     0     0     1     0     1
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     0     0     0     0     1     1     0     1     0     0     0     0
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     1     0     1     0     0     0     1     0     1     0     0     1
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]     0     0     0     0     1     0     1     0     0     0     1     0
     [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,58] [,59] [,60] [,61] [,62]
[1,]     0     1     0     0     0     1     0     0     0     1     1     0
     [,63] [,64] [,65] [,66] [,67] [,68] [,69] [,70] [,71] [,72] [,73] [,74]
[1,]     0     0     1     0     0     0     0     0     0     0     1     0
     [,75] [,76] [,77] [,78] [,79] [,80] [,81] [,82] [,83] [,84] [,85] [,86]
[1,]     1     0     0     0     1     1     0     0     0     0     1     0
     [,87] [,88] [,89] [,90] [,91] [,92] [,93] [,94] [,95] [,96] [,97] [,98]
[1,]     0     1     0     0     0     0     1     1     0     0     1     0
     [,99] [,100]
[1,]     0      1

$exact$objval
[1] 23551

$exact$slack
     [,1] [,2] [,3] [,4] [,5]
[1,]  155  100    7   43   23


$relaxed
$relaxed$solution
     [,1] [,2] [,3] [,4] [,5]      [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13]
[1,]    0    0    0    0    1 0.2218418    0    0    0     0     1     1     0
     [,14] [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25]
[1,]     1     0     0     0     0     1     0     0     1     0     0     0
     [,26] [,27] [,28] [,29]     [,30] [,31] [,32] [,33] [,34] [,35] [,36]
[1,]     0     0     0     1 0.5167232     0     0     1     0     1     0
     [,37] [,38] [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48]
[1,]     0     1     0     0     0     0     1     0     1     0     0     0
     [,49] [,50] [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,58] [,59] [,60]
[1,]     1     0     0     1     0     0     0     1     0     0     0     1
         [,61] [,62] [,63] [,64] [,65] [,66] [,67] [,68] [,69] [,70] [,71]
[1,] 0.7805382     0     0     0     1     0     0     0     0     1     0
     [,72] [,73] [,74] [,75] [,76] [,77] [,78] [,79] [,80] [,81]     [,82]
[1,]     0     1     0     1     0     0     0     1     1     0 0.3844757
     [,83] [,84] [,85] [,86] [,87] [,88] [,89] [,90] [,91] [,92] [,93] [,94]
[1,]     0     0     1     0     0     1     0     0     0     0     1     1
     [,95] [,96] [,97] [,98] [,99]    [,100]
[1,]     0     0     1     0     0 0.3778225

$relaxed$objval
[1] 23895.83

$relaxed$slack
             [,1] [,2]         [,3]          [,4]          [,5]
[1,] 3.637979e-12    0 3.637979e-12 -1.818989e-12 -1.818989e-12


$rounded
$rounded$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    0    0    0    0    0    0    0    0    0     0     1     0     0     0
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     0     0     0     0     1     0     0     0     0     0     0     0
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     0     0     1     0     0     0     1     0     1     0     0     0
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]     0     0     0     0     1     0     1     0     0     0     1     0
     [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,58] [,59] [,60] [,61] [,62]
[1,]     0     1     0     0     0     1     0     0     0     1     0     0
     [,63] [,64] [,65] [,66] [,67] [,68] [,69] [,70] [,71] [,72] [,73] [,74]
[1,]     0     0     1     0     0     0     0     1     0     0     1     0
     [,75] [,76] [,77] [,78] [,79] [,80] [,81] [,82] [,83] [,84] [,85] [,86]
[1,]     1     0     0     0     1     1     0     

##### mknapcb1.4

In [111]:
mknapcb1.4 <- ReadMultidimensionalKnapsack(paste0(dataset.folder.multi, "mknapcb1_4.dat"))

In [112]:
GenerateMultidimensionalKnapsackSolutions(mknapcb1.4)

$exact
$exact$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    1    1    0    1    0    1    0    0    1     0     0     1     0     1
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     0     0     0     0     0     0     0     0     1     0     1     0
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     0     1     0     0     1     0     0     0     1     1     0     0
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]     0     0     0     0     1     0     0     0     0     0     0     1
     [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,58] [,59] [,60] [,61] [,62]
[1,]     0     0     0     1     1     1     1     0     1     0     0     1
     [,63] [,64] [,65] [,66] [,67] [,68] [,69] [,70] [,71] [,72] [,73] [,74]
[1,]     0     1     0     0     0     0     0     0     0     0     0     0
     [,75] [,76] [,77] [,78] [,79] [,80] [,81] [,82] [,83] [,84] [,85] [,86]
[1,]     1     0     0     0     1     1     0     0     0     0     0     0
     [,87] [,88] [,89] [,90] [,91] [,92] [,93] [,94] [,95] [,96] [,97] [,98]
[1,]     1     0     0     0     0     0     0     0     1     1     0     0
     [,99] [,100]
[1,]     0      0

$exact$objval
[1] 23534

$exact$slack
     [,1] [,2] [,3] [,4] [,5]
[1,]   84   31  164   83    3


$relaxed
$relaxed$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12]     [,13]
[1,]    1    1    0    1    0    1    0    0    1     0     0     1 0.5925463
     [,14] [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25]
[1,]     1     0     0     0     0     0     0     0     0     1     0     1
     [,26] [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37]
[1,]     0     0     1     0     0     1     0     0     0     1     1     0
     [,38] [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49]
[1,]     0     0     0     0     0     1     0     0     0     0     0     0
         [,50] [,51] [,52] [,53] [,54] [,55]    [,56] [,57] [,58] [,59] [,60]
[1,] 0.8998539     0     0     0     1     1 0.935359     1     0     0     0
     [,61] [,62] [,63] [,64]     [,65] [,66] [,67] [,68] [,69] [,70] [,71]
[1,]     0     1     0     1 0.5488564     0     0     0     1     0     1
     [,72] [,73] [,74] [,75] [,76] [,77] [,78] [,79] [,80] [,81] [,82] [,83]
[1,]     0     0     0     0     0     1     0     1     1     0     0     0
     [,84] [,85] [,86]     [,87] [,88] [,89] [,90] [,91] [,92] [,93] [,94]
[1,]     0     0     0 0.7578135     0     0     0     0     0     0     0
     [,95] [,96] [,97] [,98] [,99] [,100]
[1,]     1     1     0     0     0      0

$relaxed$objval
[1] 23724.14

$relaxed$slack
              [,1]          [,2] [,3]          [,4]         [,5]
[1,] -5.456968e-12 -7.275958e-12    0 -1.818989e-12 -1.63709e-11


$rounded
$rounded$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    1    1    0    1    0    0    0    0    1     0     0     1     0     1
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     0     0     0     0     0     0     0     0     1     0     1     0
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     0     1     0     0     1     0     0     0     0     1     0     0
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]     0     0     0     0     1     0     0     0     0     0     0     0
     [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,58] [,59] [,60] [,61] [,62]
[1,]     0     0     0     0     1     0     1     0     0     0     0     1
     [,63] [,64] [,65] [,66] [,67] [,68] [,69] [,70] [,71] [,72] [,73] [,74]
[1,]     0     1     0     0     0     0     0     0     1     0     0     0
     [,75] [,76] [,77] [,78] [,79] [,80] [,81] [,82] [,83] [,84] [,85] [,86]
[1,]     0     0     1     0     0     1     0     

##### mknapcb1.5

In [113]:
mknapcb1.5 <- ReadMultidimensionalKnapsack(paste0(dataset.folder.multi, "mknapcb1_5.dat"))

In [114]:
GenerateMultidimensionalKnapsackSolutions(mknapcb1.5)

$exact
$exact$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    0    1    0    0    1    0    0    0    1     0     0     0     0     1
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     0     0     0     1     0     0     0     0     0     0     0     0
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     0     0     1     0     0     0     0     0     1     0     0     0
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]     0     0     1     0     0     0     1     0     1     0     0     1
     [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,58] [,59] [,60] [,61] [,62]
[1,]     1     0     0     0     1     1     1     0     0     0     1     1
     [,63] [,64] [,65] [,66] [,67] [,68] [,69] [,70] [,71] [,72] [,73] [,74]
[1,]     1     0     0     1     1     1     0     0     0     1     0     0
     [,75] [,76] [,77] [,78] [,79] [,80] [,81] [,82] [,83] [,84] [,85] [,86]
[1,]     0     0     0     0     0     1     0     1     0     0     0     0
     [,87] [,88] [,89] [,90] [,91] [,92] [,93] [,94] [,95] [,96] [,97] [,98]
[1,]     0     1     0     0     0     0     1     0     0     1     1     0
     [,99] [,100]
[1,]     1      1

$exact$objval
[1] 23991

$exact$slack
     [,1] [,2] [,3] [,4] [,5]
[1,]    6   71   59   66   78


$relaxed
$relaxed$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12]     [,13]
[1,]    0    0    1    0    1    0    0    0    1     0     0     0 0.8020439
     [,14] [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25]
[1,]     1     0     0     0     1     0     0     0     0     0     0     0
     [,26] [,27] [,28] [,29] [,30] [,31] [,32]     [,33] [,34] [,35] [,36]
[1,]     0     0     0     1     0     0     0 0.3924374     0     1     0
     [,37] [,38] [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48]
[1,]     0     0     0     0     1     0     0     0     1     0     1     0
     [,49] [,50] [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,58] [,59] [,60]
[1,]     0     1     1     0     0     0     1     1     1     0     0     0
     [,61] [,62] [,63] [,64] [,65] [,66]     [,67] [,68] [,69] [,70] [,71]
[1,]     1     1     1     0     0     1 0.7152418     1     0     0     0
     [,72] [,73] [,74] [,75] [,76] [,77] [,78] [,79] [,80] [,81] [,82] [,83]
[1,]     0     0     0     0     0     0     0     1     1     0     1     0
     [,84] [,85] [,86] [,87] [,88] [,89] [,90] [,91] [,92] [,93] [,94] [,95]
[1,]     0     0     0     0     1     0     0     0     0     1     0     0
         [,96]     [,97] [,98] [,99] [,100]
[1,] 0.8803146 0.1602418     0     1      1

$relaxed$objval
[1] 24223.03

$relaxed$slack
              [,1]         [,2]          [,3]          [,4]          [,5]
[1,] -1.818989e-12 7.275958e-12 -1.455192e-11 -3.637979e-12 -3.637979e-12


$rounded
$rounded$solution
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    0    0    0    0    1    0    0    0    1     0     0     0     0     1
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     0     0     0     1     0     0     0     0     0     0     0     0
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]     0     0     1     0     0     0     0     0     1     0     0     0
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]     0     0     1     0     0     0     1     0     0     0     0     1
     [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,58] [,59] [,60] [,61] [,62]
[1,]     0     0     0     0     0     0     1     0     0     0     1     0
     [,63] [,64] [,65] [,66] [,67] [,68] [,69] [,70] [,71] [,72] [,73] [,74]
[1,]     0     0     0     0     0     1     0     0     0     0     0     0
     [,75] [,76] [,77] [,78] [,79] [,80] [,81] [,82] [,83] [,84] [,85] [,86]
[1,]     0     0     0     0   

## Problema de Empaquetamiento

#### Descripción:
TODO

### Empaquetamiento Unidimensional

#### Descripción:
TODO

#### Modelo:
TODO

In [115]:
OneDimBinPackingExact <- function(p, ...) {    
    
}

#### Heurísticas:
TODO

In [116]:
OneDimBinPackingFFD <- function(p, ...) {
    
}

### Ejemplos

In [117]:
GenerateOneDimBinPackingSolutions <- function(p, ...) {
    sol <- list()
    
    #TODO(@garciparedes): Implement Bin Packing Solvers
    
    return(sol)
}

##### u120.00

In [118]:
u120.00 <- ReadBinPacking(paste0(dataset.folder.binpacking, "u120_00.dat"))

##### u120.01

In [119]:
u120.01 <- ReadBinPacking(paste0(dataset.folder.binpacking, "u120_01.dat"))

##### u120.02

In [120]:
u120.02 <- ReadBinPacking(paste0(dataset.folder.binpacking, "u120_02.dat"))

##### u250.00

In [121]:
u250.00 <- ReadBinPacking(paste0(dataset.folder.binpacking, "u250_00.dat"))

##### u250.01

In [122]:
u250.01 <- ReadBinPacking(paste0(dataset.folder.binpacking, "u250_01.dat"))

##### u250.02

In [123]:
u250.02 <- ReadBinPacking(paste0(dataset.folder.binpacking, "u250_02.dat"))

##### u500.00

In [124]:
u500.00 <- ReadBinPacking(paste0(dataset.folder.binpacking, "u500_00.dat"))

##### u500.01

In [125]:
u500.01 <- ReadBinPacking(paste0(dataset.folder.binpacking, "u500_01.dat"))

##### u500.02

In [126]:
u500.02 <- ReadBinPacking(paste0(dataset.folder.binpacking, "u500_02.dat"))

## Conclusiones

TODO

## Referencias

  * [TRC13] Team, R.C., 2013. R: A language and environment for statistical computing.
  * [GP18] Sergio García Prado. Programación Entera: Heurísticas, 2018. [github.com/garciparedes/integer-programming-heuristics](https://github.com/garciparedes/integer-programming-heuristics).
  * [SA18] Jesús Sáez Aguado. Programación Entera, 2017/18. Facultad de Ciencias: Departamento de Estadística e Investigación Operativa.
  * [THBSST17] Theussl, S., Hornik, K., Buchta, C., Schwendinger, F., Schuchardt, H. and Theussl, M.S., 2017. Package ‘Rglpk’.
  